# Step 4 - Points of interest based bicycle network generation
## Project: Growing Urban Bicycle Networks

This notebook follows the transit-oriented development approach of palominos2020ica or a grid approach and applies cardillo2006spp: Take the greedy triangulation between railway/underground stations (or other points of interest created in 02_prepare_pois). This is the cold start bicycle network generation process which creates bicycle networks from scratch.

TODO

- ltns un-prioritised growth to compare against ltn prioritsed  X DONE
- clean triangulation for concave shapes X DONE
- methods plotting for removing slivers etc
- cycle network investment within LTNs, converting pedestrain streets so that you can cycle through. Could have a big impact on the lcc size X Done in script 03
- can't take multiple places as an input (my bad coding skills...), need to be able to take several places at a time
- save outputs as geopackages

In [ ]:
# import libraries
from src import utils
PATH = utils.PATH # shortening the var name so that we don't have to change it below

# System
import copy
import csv
import os
import dill as pickle
from collections import defaultdict
import pprint
pp = pprint.PrettyPrinter(indent=4)
from tqdm.notebook import tqdm
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from copy import deepcopy
import yaml
import json

# Math/Data
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Network
import networkx as nx

# Plotting
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import animation # for gifs
from matplotlib.collections import LineCollection


# Geo
import osmnx as ox
ox.settings.log_file = True
ox.settings.requests_timeout = 300
ox.settings.logs_folder = PATH["logs"]
import fiona
import shapely
from haversine import haversine
from shapely.geometry import Polygon
import geopandas as gpd
import json
import momepy

### Parameters

In [ ]:
debug = False
gifs = True

In [ ]:
params = yaml.load(
    open("../parameters/parameters.yml"), 
    Loader=yaml.FullLoader)
osmnxparameters = json.load(open("../parameters/osmnxparameters.json", "r"))
plotparam = json.load(open("../parameters/plotparam.json", "r"))
plotparam_analysis = json.load(open("../parameters/plotparam_analysis.json", "r"))

# load cities
cities = utils.load_cities(PATH, debug)

### Network weighting tags

In [ ]:
tag_lts = json.load(open("../parameters/tag_lts.json", "r"))
distance_cost = json.load(open("../parameters/distance_cost.json", "r"))

# Routing 

## Setup for growth

The function and code below currently routes between the edges of neighbourhoods, rather than from a single point to a single point. We then join the neighbourhoods up first, before considering the wider area. This wider area is derived from hexagonal tesslleations within the city boundaries.

In [ ]:
# setup

def csv_to_ox(p, placeid, parameterid):
    '''
    Load graph from csv files (nodes and edge)
    Include OSMID, length, highway, x, y attributes
    '''

    prefix = placeid + '_' + parameterid
    compress = check_extract_zip(p, prefix)
    
    with open(p + prefix + '_edges.csv', 'r') as f:
        header = f.readline().strip().split(",")
        lines = []
        for line in csv.reader(f, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True):
            line_list = [c for c in line]
            osmid = str(eval(line_list[header.index("osmid")])[0]) if isinstance(eval(line_list[header.index("osmid")]), list) else line_list[header.index("osmid")]
            length = str(eval(line_list[header.index("length")])[0]) if isinstance(eval(line_list[header.index("length")]), list) else line_list[header.index("length")]
            highway = line_list[header.index("highway")]
            if highway.startswith("[") and highway.endswith("]"):
                highway = highway.strip("[]").split(",")[0].strip(" '")
            line_string = f"{line_list[header.index('u')]} {line_list[header.index('v')]} {osmid} {length} {highway}"
            lines.append(line_string)
        G = nx.parse_edgelist(lines, nodetype=int, data=(("osmid", int), ("length", float), ("highway", str)), create_using=nx.MultiDiGraph)
    
    with open(p + prefix + '_nodes.csv', 'r') as f:
        header = f.readline().strip().split(",")
        values_x = {}
        values_y = {}
        for line in csv.reader(f, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True):
            line_list = [c for c in line]
            osmid = int(line_list[header.index("osmid")])
            values_x[osmid] = float(line_list[header.index("x")])
            values_y[osmid] = float(line_list[header.index("y")])
        nx.set_node_attributes(G, values_x, "x")
        nx.set_node_attributes(G, values_y, "y")
    
    if compress:
        os.remove(p + prefix + '_nodes.csv')
        os.remove(p + prefix + '_edges.csv')
    return G



### Load data

G_caralls - no addtional weighting

G_weighted - for routing. lts applied + zero bike infrastucutre cost

G_investment distance - for finding distances between points in abstract (greedy_gdf) graph. cycle infrastucre has no cost, everything else has default distance

greedy_gdf[distance] - the investment length (routed length - any infrastucture)

GT "length" - actual distance between points

In [ ]:
# Set up data structures
locations = {}
G_caralls = {}
G_caralls_simplified = {}
G_weighteds = {}
G_investment_distances = {}
G_defaults = {}
G_biketracks = {}
G_biketracks_no_ltn = {}
ltn_nodes_dict = {}
tess_nodes_dict = {}
combined_nodes_dict = {}
all_centroids_dict = {}
edge_betweenness_dict = {}
ebc_ltn_dict = {}
ebc_other_dict = {}
ebc_all_dict = {}
shortest_paths_ltn_pairs_dict = {}
shortest_paths_other_pairs_dict = {}
shortest_paths_all_pairs_dict = {}
shortest_paths_ltn_dict = {}
shortest_paths_other_dict = {}
shortest_paths_all_dict = {}
combined_node_pairs_dict = {}
greedy_gdfs_dict = {}
tess_gdfs_dict = {}
ltn_gdfs_dict = {}
greedy_triangulation_ltns_dict = {}
ltn_node_pairs_dict = {}
exit_points_dict = {}
greedy_nx_dict = {}

parameterinfo = osmnxparameters['carall']

for scenario in params["scenarios"]:
    for placeid in cities.keys():
        print(f"Processing: {placeid} - {scenario}")

        data_path = os.path.join(PATH["data"], placeid, scenario)

        # Load road graph
        G_carall = utils.csv_to_ox(data_path + "/", placeid, 'biketrackcarall')
        G_carall.graph["crs"] = 'epsg:4326'  # Needed for compatibility with GeoDataFrames
        G_caralls[(placeid, scenario)] = G_carall

        # Make graph copies for different weighting strategies
        G_weighted = copy.deepcopy(G_carall)
        G_investment_distance = copy.deepcopy(G_carall)
        G_default = copy.deepcopy(G_carall)

        # Apply LTS weighting
        for u, v, key, data in G_weighted.edges(keys=True, data=True):
            highway = data.get('highway')
            lts = tag_lts.get(highway, 1)
            G_weighted[u][v][key]['length'] *= lts

        # Apply investment cost weighting
        for u, v, key, data in G_investment_distance.edges(keys=True, data=True):
            highway = data.get('highway')
            distance = distance_cost.get(highway, 1)
            G_investment_distance[u][v][key]['length'] *= distance

        # Convert to undirected
        G_weighteds[(placeid, scenario)] = G_weighted.to_undirected()
        G_investment_distances[(placeid, scenario)] = G_investment_distance.to_undirected()
        G_defaults[(placeid, scenario)] = G_default.to_undirected()

        # Load biketrack graph
        gpkg_path = os.path.join(data_path, f"{placeid}_biketrack.gpkg")
        G_bike = utils.ox_gpkg_to_graph(gpkg_path)
        G_bike.remove_nodes_from(list(nx.isolates(G_bike)))
        G_biketracks[(placeid, scenario)] = G_bike

        # Load biketrack graph without LTNs
        gpkg_path_no_ltn = os.path.join(data_path, f"{placeid}_biketrack_no_ltn.gpkg")
        G_bike_no_ltn = utils.ox_gpkg_to_graph(gpkg_path_no_ltn)
        G_bike_no_ltn.remove_nodes_from(list(nx.isolates(G_bike_no_ltn)))
        G_biketracks_no_ltn[(placeid, scenario)] = G_bike_no_ltn


In [ ]:
for scenario in params["scenarios"]:
    for placeid, placeinfo in tqdm(cities.items(), desc=f"Cities ({scenario})"):
        print(f"Processing: {placeid} - {scenario}")

        # Load Tesselation POIs (hard coded for now)
        with open(os.path.join(PATH["data"], placeid, scenario, f"{placeid}_poi_tessellation_nnidsbikeall.csv")) as f:
            tessellation_nnids = [int(line.rstrip()) for line in f]

        # Load LTN POIs
        if placeinfo["nominatimstring"] != '':
            location = ox.geocoder.geocode_to_gdf(placeinfo["nominatimstring"])
            if location.geometry[0].geom_type == 'MultiPolygon':
                location = location.explode(index_parts=False).reset_index(drop=True)
            location = utils.fill_holes(utils.extract_relevant_polygon(placeid, shapely.geometry.shape(location['geometry'][0])))
        else:
            # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
            shp = fiona.open(os.path.join(PATH["data"], placeid, scenario, f"{placeid}.shp"))
            first = next(iter(shp))
            try:
                location = Polygon(shapely.geometry.shape(first['geometry']))  # If shape file is given as linestring
            except:
                location = shapely.geometry.shape(first['geometry'])

        locations[(placeid, scenario)] = location

        G_caralls[(placeid, scenario)] = utils.csv_to_ox_highway(PATH["data"] + placeid + "/" + scenario + "/", placeid, 'biketrackcarall')
        G_caralls[(placeid, scenario)].graph["crs"] = 'epsg:4326'  # Needed for OSMNX's graph_to_gdfs in utils_graph.py
        G_caralls_simplified[(placeid, scenario)] = utils.csv_to_ox(PATH["data"] + placeid + "/" + scenario + "/", placeid, 'biketrackcarall_simplified')
        G_caralls_simplified[(placeid, scenario)].graph["crs"] = 'epsg:4326'  # Needed for OSMNX's graph_to_gdfs in utils_graph.py

        print(f"{placeid}: Loading and moving POIs")
        # Get the carall graph and location geometry
        location = locations[(placeid, scenario)]
        G_carall = G_caralls_simplified[(placeid, scenario)]

        # Load neighbourhoods and create GeoDataFrame for centroids
        neighbourhoods = utils.load_neighbourhoods(os.path.join(PATH["data"], placeid, scenario))
        all_centroids = gpd.GeoDataFrame(columns=['neighbourhood_id', 'geometry'], crs='EPSG:4326')  

        # load tesselation points 
        tess_nodes = gpd.read_file(os.path.join(PATH["data"], placeid, scenario, f"{placeid}_poi_tessellation.gpkg"))
        # load ltns and get exit points of ltns
        if scenario != "no_ltn_scenario":
            exit_points = utils.get_exit_nodes(neighbourhoods, G_biketracks[(placeid, scenario)])
            exit_points_dict[(placeid, scenario)] = exit_points.copy()
            if params["export"]:
                file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_exit_points.gpkg")
                os.makedirs(os.path.dirname(file_path), exist_ok=True)
                exit_points.to_file(file_path, driver="GPKG")


        unique_id = 0  # Counter for unique IDs across neighbourhoods
        for name, gdf in neighbourhoods.items():  # Process each neighbourhood GeoDataFrame to get centroids, exit points, and neighbourhood IDs
            if gdf.empty:
                print(f"Warning: The GeoDataFrame for {name} is empty. Skipping...")
                continue
            print(f"Processing neighbourhoods in: {name}")

            # Assign a unique ID to each neighbourhood in the GeoDataFrame to reference throughout
            gdf['neighbourhood_id'] = range(unique_id, unique_id + len(gdf))
            if debug:
                print(f"Assigned neighbourhood_ids from {unique_id} to {unique_id + len(gdf) - 1} for {name}")

            # Get centroids to inherit 'neighbourhood_id'
            centroids_gdf = utils.get_neighbourhood_centroids(gdf)
            all_centroids = pd.concat([all_centroids, centroids_gdf], ignore_index=True)
            unique_id += len(gdf)  # Increment by the number of neighbourhoods processed

        # Snap centroids to the closest nodes in the street network
        neighbourhood_nnids = set()
        for g in all_centroids['geometry']:
            n = ox.distance.nearest_nodes(G_carall, g.x, g.y)
            if n not in neighbourhood_nnids and haversine((g.y, g.x), (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]), unit="m") <= params["snapthreshold"]:
                neighbourhood_nnids.add(n)
        # Add nearest_node column to all_centroids by finding the nearest node for each centroid geometry
        all_centroids['nearest_node'] = all_centroids['geometry'].apply(
            lambda g: ox.distance.nearest_nodes(G_carall, g.x, g.y))  # We now have all_centroids with 'neighbourhood_id', 'geometry', 'nearest_node' columns
        all_centroids['osmid'] = all_centroids['nearest_node']
        ltn_nodes = all_centroids
        all_centroids_dict[(placeid, scenario)] = all_centroids.copy()

        # add nearest node ID from G_carall 
        tess_nn = utils.get_nearest_nodes_to_gdf(G_carall, tess_nodes)
        tess_nodes['osmid'] = tess_nn
        tess_nodes_dict[(placeid, scenario)] = tess_nodes
        if scenario != "no_ltn_scenario":
            ltn_nodes = all_centroids
            ltn_nn = utils.get_nearest_nodes_to_gdf(G_carall, ltn_nodes)
            ltn_nodes['osmid'] = ltn_nn
            combined_nodes = pd.concat([tess_nodes, ltn_nodes], ignore_index=True)
            combined_nodes_dict[(placeid, scenario)] = combined_nodes
            ltn_nodes_dict[(placeid, scenario)] = ltn_nodes
        else:
            combined_nodes = tess_nodes
            combined_nodes_dict[(placeid, scenario)] = tess_nodes

        # save them
        out_dir = os.path.join(PATH["data"], placeid, scenario)
        tess_nodes.to_file(os.path.join(out_dir, f"{placeid}_tess_points.gpkg"), driver="GPKG")
        if scenario != "no_ltn_scenario":
            ltn_nodes.to_file(os.path.join(out_dir, f"{placeid}_ltn_points.gpkg"), driver="GPKG")
        combined_nodes.to_file(os.path.join(out_dir, f"{placeid}_combined_points.gpkg"), driver="GPKG")
        print(f"Saved tessellation and LTN points for {placeid} in {out_dir}")

## Create triangulations

Build LTN triangulation

In [ ]:
# Produce triangulation for LTN nodes
# only needed to set up full trianuglation, not needed later.
# ltn nodes are not ordered (nor do they need to be at this stage)

for scenario in params["scenarios"]:
    if scenario == "no_ltn_scenario":
        print(f"Skipping triangulation for {scenario} as it does not contain LTNs.")
        continue  # Skip triangulation for 'no_ltn_scenario'

    for placeid in cities:
        print(f"Triangulating LTN nodes for {placeid} ({scenario})")
        ltn_nodes = ltn_nodes_dict[(placeid, scenario)]  
        greedy_triangulation_ltns_gdf = utils.greedy_triangulation_ltns(ltn_nodes)
        ltn_node_pairs = utils.get_ltn_node_pairs(ltn_nodes, greedy_triangulation_ltns_gdf)
        greedy_triangulation_ltns_dict[(placeid, scenario)] = greedy_triangulation_ltns_gdf
        ltn_node_pairs_dict[(placeid, scenario)] = ltn_node_pairs


        if params["export"]:
            file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_greedy_triangulation_ltns.gpkg")
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            greedy_triangulation_ltns_gdf.to_file(file_path, driver="GPKG")



Build full triangulation using greedy triangulation

In [ ]:
for scenario in params["scenarios"]:
    if scenario == "no_ltn_scenario":
        tess_nodes = tess_nodes_dict[(placeid, scenario)]
        greedy_gdf, tess_gdf = utils.build_greedy_triangulation_no_ltns(tess_nodes)
        max_length = greedy_gdf['distance'].sum()
        greedy_gdfs_dict[(placeid, scenario)] = greedy_gdf
        tess_gdfs_dict[(placeid, scenario)] = tess_gdf
        continue
    ltn_nodes = ltn_nodes_dict[(placeid, scenario)]
    tess_nodes = tess_nodes_dict[(placeid, scenario)]

    greedy_gdf, ltn_gdf, tess_gdf = utils.build_greedy_triangulation(ltn_nodes, tess_nodes)
    max_length = greedy_gdf['distance'].sum()
    greedy_gdfs_dict[(placeid, scenario)] = greedy_gdf
    tess_gdfs_dict[(placeid, scenario)] = tess_gdf
    ltn_gdfs_dict[(placeid, scenario)] = ltn_gdf


Clean up greedy triangulation to remove silver triangles, and unrealisticly long links.
- Removing silvers removes links which would pass along the same route (thus essentially duplicating themselves)
- Links longer than 5km are not realistic links to be making, they almost always link rural to rural over routes which can be made

In [ ]:
for scenario in params["scenarios"]:
    for placeid in cities:
        greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]  
        # find very long links
        greedy_gdf = greedy_gdf[greedy_gdf["distance"] <= 5000]
        # find slivers and remove the longest edge from them
        fas = momepy.FaceArtifacts(greedy_gdf) 
        # run FaceArtifacts (cf https://docs.momepy.org/en/stable/api/momepy.FaceArtifacts.html)
        fas.polygons = fas.polygons.set_crs(greedy_gdf.crs)
        threshold = 10
        polys = fas.polygons.copy()
        slivers = polys[polys['face_artifact_index'] < threshold]
        slivers["centroid"] = slivers.geometry.centroid
        centroids = slivers["centroid"]
        bounding = utils.find_bounding_lines(centroids, greedy_gdf)
        longest_lines = []
        for pt_idx, line_idxs in bounding.items():
            if not line_idxs:
                longest_lines.append(None)  
                continue
            lines = greedy_gdf.loc[line_idxs]
            # Find the longest line
            max_line = lines.loc[lines["distance"].idxmax()]
            longest_lines.append(max_line.name)
        lines_to_drop = [idx for idx in longest_lines if idx is not None]
        greedy_gdf = greedy_gdf.drop(index=lines_to_drop)
        # Save back the cleaned graph
        greedy_gdfs_dict[(placeid, scenario)] = greedy_gdf


In [ ]:
if params["methods_plotting"]:
    greedy_gdf

### Build Full Triangulation (cont.)

In [ ]:
# get node pairs of ltn-to-tess, tess-to-ltn, and tess-to-tess
for scenario in params["scenarios"]:
    if scenario == "no_ltn_scenario":
        combined_nodes = combined_nodes_dict[(placeid, scenario)]
        greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
        combined_node_pairs = utils.get_node_pairs_no_ltn(combined_nodes, greedy_gdf)
        combined_node_pairs_dict[(placeid, scenario)] = combined_node_pairs
        continue  # Skip non-LTN scenarios
    for placeid in cities:
        combined_nodes = combined_nodes_dict[(placeid, scenario)]
        greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
        ltn_node_pairs = ltn_node_pairs_dict[(placeid, scenario)]
        # get node pairs of ltn-to-tess, tess-to-ltn, and tess-to-tess
        combined_node_pairs = utils.get_node_pairs(combined_nodes, greedy_gdf)
        # remove ltn-ltn pairs 
        combined_node_pairs = [pair for pair in combined_node_pairs if pair not in ltn_node_pairs]
        combined_node_pairs_dict[(placeid, scenario)] = combined_node_pairs

Apply Level of Traffic Stress to triangulation distances

In [ ]:
for scenario in params["scenarios"]:
    for placeid in cities:
        greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
        G_weighted = G_weighteds[(placeid, scenario)]
        G_default = G_defaults[(placeid, scenario)]
        greedy_gdf['sp_lts_route'] = greedy_gdf.apply(lambda row: nx.shortest_path(G_weighted, source=row['start_osmid'], target=row['end_osmid'], weight='length'),axis=1)
        greedy_gdf['sp_lts_distance'] = greedy_gdf['sp_lts_route'].apply(lambda route: sum(G_weighted[u][v][0]['length'] for u, v in zip(route[:-1], route[1:])))
        # find normal network distance for directness analysis
        #G_default = copy.deepcopy(G_weighted) ## remove, G_weighted has lengths as zero so cannot be deweighted
        #deweight_edges(G_default, tag_lts)
        greedy_gdf['sp_true_distance'] = greedy_gdf['sp_lts_route'].apply(lambda route: sum(G_default[u][v][0]['length'] for u, v in zip(route[:-1], route[1:])))
        greedy_gdf['eucl_dist'] = greedy_gdf['distance']  # save for directness analysis
        greedy_gdfs_dict[(placeid, scenario)] = greedy_gdf

In [ ]:
for scenario in params["scenarios"]:
    for placeid in cities:
        greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
        # Create the graph from the triangulation GeoDataFrame
        greedy_nx = nx.Graph()
        for _, row in greedy_gdf.iterrows():
            start = row['start_osmid']
            end = row['end_osmid']
            sp_lts_distance = row['sp_lts_distance']
            greedy_nx.add_edge(start, end, geometry=row['geometry'], sp_lts_distance=sp_lts_distance)
        greedy_nx_dict[(placeid, scenario)] = greedy_nx


Get shortest paths

In [ ]:
for scenario in params["scenarios"]:
    for placeid in cities:
        greedy_nx = greedy_nx_dict[(placeid, scenario)]
        combined_node_pairs = combined_node_pairs_dict[(placeid, scenario)]

        if scenario != "no_ltn_scenario":
            # for ltn prioirty 
            ltn_node_pairs = ltn_node_pairs_dict[(placeid, scenario)]

            shortest_paths_ltn = []
            shortest_paths_other = []
            for node1, node2 in ltn_node_pairs:
                try:
                    path = nx.shortest_path(greedy_nx, source=node1, target=node2, weight='sp_lts_distance')
                    shortest_paths_ltn.append(path)
                except nx.NetworkXNoPath:
                    print(f"No path between {node1} and {node2}")
            for node1, node2 in combined_node_pairs:
                try:
                    path = nx.shortest_path(greedy_nx, source=node1, target=node2, weight='sp_lts_distance')
                    shortest_paths_other.append(path)
                except nx.NetworkXNoPath:
                    print(f"No path between {node1} and {node2}")
            shortest_paths_ltn_dict[(placeid, scenario)] = shortest_paths_ltn
            shortest_paths_other_dict[(placeid, scenario)] = shortest_paths_other
            

        # for non-ltn priority
        shortest_paths_all = []
        for node1, node2 in combined_node_pairs:
            try:
                path = nx.shortest_path(greedy_nx, source=node1, target=node2, weight='sp_lts_distance')
                shortest_paths_all.append(path)
            except nx.NetworkXNoPath:
                print(f"No path between {node1} and {node2}")
        shortest_paths_all_dict[(placeid, scenario)] = shortest_paths_all


Find path values for ranking: Get order using sum of { (ebc of edge * (length of edge/sum of length of edges) ) } / number-of-edges.

Distance used in these functions is the routed lts distance in meters

In [ ]:
# Calculate edge betweenness centrality

for scenario in params["scenarios"]:
    for placeid in cities:
        greedy_nx = greedy_nx_dict[(placeid, scenario)]
        combined_node_pairs = combined_node_pairs_dict[(placeid, scenario)]
        shortest_paths_all = shortest_paths_all_dict[(placeid, scenario)]

        edge_betweenness = nx.edge_betweenness_centrality(greedy_nx, weight='sp_lts_distance')
        edge_betweenness_dict[(placeid, scenario)] = edge_betweenness

        if scenario != "no_ltn_scenario":
            ltn_node_pairs = ltn_node_pairs_dict[(placeid, scenario)]
            shortest_paths_ltn = shortest_paths_ltn_dict[(placeid, scenario)]
            shortest_paths_other = shortest_paths_other_dict[(placeid, scenario)]

            # find path values
            ebc_ltn = utils.get_sp_ebc_weights(ltn_node_pairs, shortest_paths_ltn, greedy_nx, edge_betweenness)
            ebc_other = utils.get_sp_ebc_weights(combined_node_pairs, shortest_paths_other, greedy_nx, edge_betweenness)

            # order by betweenness path value
            ebc_ltn = dict(sorted(ebc_ltn.items(), key=lambda item: item[1], reverse=True))
            ebc_other = dict(sorted(ebc_other.items(), key=lambda item: item[1], reverse=True))

            # set up shortest paths for looping through in budget adjustment
            shortest_paths_ltn = {
                (path[0], path[-1]): [(path[i], path[i+1]) for i in range(len(path)-1)]
                for path in shortest_paths_ltn
            }
            shortest_paths_other = {
                (path[0], path[-1]): [(path[i], path[i+1]) for i in range(len(path)-1)]
                for path in shortest_paths_other
            }

            ebc_ltn_dict[(placeid, scenario)] = ebc_ltn
            ebc_other_dict[(placeid, scenario)] = ebc_other
            shortest_paths_ltn_pairs_dict[(placeid, scenario)] = shortest_paths_ltn
            shortest_paths_other_pairs_dict[(placeid, scenario)] = shortest_paths_other

        # for all paths (non-ltn priority)
        ebc_all = utils.get_sp_ebc_weights(combined_node_pairs, shortest_paths_all, greedy_nx, edge_betweenness)
        ebc_all = dict(sorted(ebc_all.items(), key=lambda item: item[1], reverse=True))
        shortest_paths_all = {
            (path[0], path[-1]): [(path[i], path[i+1]) for i in range(len(path)-1)]
            for path in shortest_paths_all
        }

        ebc_all_dict[(placeid, scenario)] = ebc_all
        shortest_paths_all_pairs_dict[(placeid, scenario)] = shortest_paths_all


### Compute routed edge lengths for abstract graph

This ensures we use our "budget" correctly. We first find the optimal route using the LTS tags per connection, then "deweight" the path to find the "true" length of the connection in meters. Existing infrastucture (bike paths etc) are not included in the "true" length, as we do not need to build these facilities.

In [ ]:
## this cell can take up to 30 mins to run
for scenario in params["scenarios"]:
    for placeid in cities:
        print(f"Computing shortest paths for {placeid} ({scenario})")
        greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
        G_weighted = G_weighteds[(placeid, scenario)]
        G_investment_distance = G_defaults[(placeid, scenario)]
        if scenario != "no_ltn_scenario":
            all_centroids = all_centroids_dict[(placeid, scenario)]
            exit_points = exit_points_dict[(placeid, scenario)]

            ## find the routed distance between edges in abstract GT
            centroid_osmids = set(all_centroids['osmid'])
            greedy_gdf['ltn_origin'] = greedy_gdf['start_osmid'].isin(centroid_osmids)
            greedy_gdf['ltn_destination'] = greedy_gdf['end_osmid'].isin(centroid_osmids)

            # Build a mapping from centroid osmid to neighbourhood_id
            osmid_to_neigh = dict(zip(all_centroids['osmid'], all_centroids['neighbourhood_id']))

            # Build a mapping from neighbourhood_id to list of exit point osmids
            neigh_to_exits = defaultdict(list)
            for idx, row in exit_points.iterrows():
                neigh_to_exits[row['neighbourhood_id']].append(row['osmid'])

        else:
            # Make these columns/empty mappings to avoid getting an error in the apply function
            greedy_gdf['ltn_origin'] = False
            greedy_gdf['ltn_destination'] = False
            osmid_to_neigh = {}
            neigh_to_exits = defaultdict(list)
        
        # Compute the shortest path and store in 'sp_route'
        sp_routes = []
        for idx, row in greedy_gdf.iterrows():
            route = utils.compute_routed_path_for_GT(row, G_weighted, osmid_to_neigh, neigh_to_exits) # <-- this seems to be very slow!?
            sp_routes.append(route)
        greedy_gdf['sp_route'] = sp_routes
        
        # Find lengths of shortest paths on G_investment_distance
        # greedy_gdf['distance'] = greedy_gdf['sp_route'].apply(lambda route: calculate_sp_route_distance(route, G_investment_distance))
        distances = []
        for route in greedy_gdf['sp_route']:
            if route is not None:
                distance = utils.calculate_sp_route_distance(route, G_investment_distance)
            else:
                distance = None
            distances.append(distance)

        del greedy_gdf['sp_route']
        greedy_gdfs_dict[(placeid, scenario)] = greedy_gdf.copy()


Now "Distance" is the length of investment required to connect two locations via the best(shortest with weighting of LTS) path

In [ ]:
for scenario in params["scenarios"]:
    for placeid in cities:
        if params["methods_plotting"]:

            print(f"Plotting for {placeid} ({scenario})")

            # Setup
            greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
            location = locations[(placeid, scenario)]
            location_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries([location]), crs="EPSG:4326")

            plot_path = PATH["plots"] + placeid + "/" + scenario + "/"

            #### PLOT EBC MAP ####
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.axis('off')
            location_gdf.plot(ax=ax, color='lightgrey', zorder=0)

            tess_nodes_dict[(placeid, scenario)].plot(ax=ax, color='red', markersize=10, zorder=2)

            if scenario != "no_ltn_scenario":
                ltn_nodes_dict[(placeid, scenario)].plot(ax=ax, color='red', markersize=10, zorder=3)

            edge_betweenness = edge_betweenness_dict[(placeid, scenario)]
            ebc_df = pd.DataFrame([
                {'start': min(u, v), 'end': max(u, v), 'betw': w}
                for (u, v), w in edge_betweenness.items()
            ])

            merged_gdf = (
                greedy_gdf
                .assign(
                    start_norm=greedy_gdf[['start_osmid', 'end_osmid']].min(axis=1),
                    end_norm=greedy_gdf[['start_osmid', 'end_osmid']].max(axis=1)
                )
                .merge(ebc_df, left_on=['start_norm', 'end_norm'], right_on=['start', 'end'], how='left')
                .to_crs(location_gdf.crs)
            )

            merged_gdf.plot(
                ax=ax, column='betw', legend=True, cmap='viridis', linewidth=2, zorder=1,
                legend_kwds={'shrink': 0.6, 'label': "Normalised Edge Betweenness Centrality"}
            )

            plt.savefig(plot_path + "greedy_tri_ebc_map.png", dpi=600, bbox_inches='tight')
            plt.close()

            #### PLOT DOUBLE GREEDY MAP ####
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.axis('off')
            location_gdf.plot(ax=ax, color='lightgrey', zorder=0)

            if scenario != "no_ltn_scenario":
                ltn_nodes_dict[(placeid, scenario)].plot(ax=ax, color='darkorange', markersize=50, zorder=3)

            greedy_gdf.to_crs(location_gdf.crs).plot(ax=ax, color='firebrick', linewidth=1, zorder=1)

            if scenario != "no_ltn_scenario":
                greedy_triangulation_ltns_dict[(placeid, scenario)].to_crs(location_gdf.crs).plot(
                    ax=ax, color='royalblue', linewidth=2, zorder=2
                )

            plt.savefig(plot_path + "greedy_tri_neighbours_map.png", dpi=600, bbox_inches='tight')
            plt.close()


In [ ]:
if params["export"]:
    for scenario in params["scenarios"]:
        for placeid in cities:
            print(f"Exporting GPKG for {placeid} ({scenario})")
            gdf = greedy_gdfs_dict[(placeid, scenario)]
            ebc = edge_betweenness_dict[(placeid, scenario)]
            ebc_df = pd.DataFrame([{'start': min(u, v), 'end': max(u, v), 'betw': w} for (u, v), w in ebc.items()])
            gdf['start_norm'] = gdf[['start_osmid', 'end_osmid']].min(axis=1) # norm means normilsed direction
            gdf['end_norm']   = gdf[['start_osmid', 'end_osmid']].max(axis=1)

            merged = (
                gdf.merge(ebc_df, left_on=['start_norm', 'end_norm'], right_on=['start', 'end'], how='left')
                .to_crs(4326)
            )
            path = os.path.join(PATH["exports_gpkg"], placeid, scenario)
            os.makedirs(path, exist_ok=True)
            merged.to_file(os.path.join(path, f"{placeid}_greedy_triangulation.gpkg"), driver="GPKG")


Find the size of a fully connected network

In [ ]:
investment_levels_dict = {}
for scenario in params["scenarios"]:
    if scenario != "current_ltn_scenario":
        continue  # Skip other scenarios
    for placeid in cities:
        gdf = greedy_gdfs_dict.get((placeid, scenario))
        if gdf is None or 'distance' not in gdf.columns:
            if debug:
                print(f"Skipping {placeid} ({scenario}): missing data.")
            continue
        total_distance = gdf['distance'].sum()
        if debug:
            print(f"{placeid} ({scenario}): {total_distance:.2f} meters")
        # Set investment max to a fully connected bicycle network and define 1% incremental steps
        points = np.linspace(0, total_distance, 101)
        investment_levels = points[1:].tolist()  # Drop 0 as we don't consider the empty network
        investment_levels_dict[(placeid, scenario)] = investment_levels

In [ ]:
# Now compute investment levels for other scenarios using same intervals
for scenario in params["scenarios"]:
    if scenario == "current_ltn_scenario":
        continue  # Skip current_ltn_scenario here
    for placeid in cities:
        gdf = greedy_gdfs_dict.get((placeid, scenario))
        if gdf is None or 'distance' not in gdf.columns:
            if debug:
                print(f"Skipping {placeid} ({scenario}): missing data.")
            continue
        total_distance = gdf['distance'].sum()
        if debug:
            print(f"{placeid} ({scenario}): {total_distance:.2f} meters")
        current_levels = investment_levels_dict.get((placeid, "current_ltn_scenario"))
        if current_levels is None:
            if debug:
                print(f"Skipping {placeid} ({scenario}): missing current scenario for step size.")
            continue

        step_size = current_levels[0]  # use investment step size from current_ltn_scenario
        investment_levels = []
        next_level = step_size
        while next_level < total_distance:
            investment_levels.append(next_level)
            next_level += step_size
        investment_levels.append(min(total_distance, next_level))  

        investment_levels_dict[(placeid, scenario)] = investment_levels

        

## EBC Growth - Set connection order (LTNs prioritised)

Greedy triangulation "distance" is the length of the shortest path distance between edges once routed on to the network, without including any length from existing infrastucture. 

This cell creates a growth order based on **betweeness centrality**. Priority is given to Low Traffic Neighbouhoods, only once LTNs have been connected are any other links added.

Create iterations of network growth:

In [ ]:
## betwenness LTN priority
## many combinations
GTs_dict = {}
GT_abstracts_dict = {}

for scenario in params["scenarios"]:
    if scenario == "no_ltn_scenario":
        print(f"Skipping scenario: {scenario}, got no LTNs")
        continue

    previous_selected_edges = set()

    GT_abstracts = []
    GT_abstracts_gdf = []
    GTs = []
    GTs_gdf = []

    global_processed_pairs = set()
    cumulative_GT_indices = set()

    G_weighted = G_weighteds[(placeid, scenario)]
    G_carall = G_caralls[(placeid, scenario)]

    greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
    shortest_paths_ltn_pairs = shortest_paths_ltn_pairs_dict[(placeid, scenario)]
    ebc_ltn = ebc_ltn_dict[(placeid, scenario)]
    shortest_paths_other_pairs = shortest_paths_other_pairs_dict[(placeid, scenario)]
    ebc_other = ebc_other_dict[(placeid, scenario)]

    all_centroids = all_centroids_dict[(placeid, scenario)]
    tess_gdf = tess_gdfs_dict[(placeid, scenario)]
    ltn_gdf = ltn_gdfs_dict[(placeid, scenario)]
    exit_points = exit_points_dict[(placeid, scenario)]

    if ltn_extra is not None and not ltn_extra.empty:
        ltn_extra_edges = ltn_extra.reset_index()[['u', 'v']]
        ltn_extra_edge_set = set(map(tuple, ltn_extra_edges.to_numpy()))
        ltn_extra_edge_set |= {(v, u) for u, v in ltn_extra_edge_set}  # make undirected
    else:
        ltn_extra_edge_set = set()


    for D in tqdm(investment_levels_dict[(placeid, scenario)], desc=f"Pruning GT abstract and routing on network for meters of investment - {scenario}"):
        GT_abstract_gdf, previous_selected_edges, connected_ltn_pairs, connected_other_pairs = utils.adjust_triangulation_to_budget_ltn_priority(
            greedy_gdf, D,
            shortest_paths_ltn_pairs,
            ebc_ltn,
            shortest_paths_other_pairs,
            ebc_other,
            previous_selected_edges,
            ltn_node_pairs_dict[(placeid, scenario)])

        remaining_edges = len(greedy_gdf) - len(previous_selected_edges)
        if debug:
            print(f"[{scenario}] Remaining edges to add: {remaining_edges}")

        GT_abstracts_gdf.append(GT_abstract_gdf)
        GT_abstract_nx = utils.gdf_to_nx_graph(GT_abstract_gdf)
        GT_abstracts.append(GT_abstract_nx)

        poipairs = list(GT_abstract_nx.edges())
        routenodepairs = [(u, v) for u, v in poipairs]

        if debug:
            print(f"[{scenario}] Routing {len(routenodepairs)} pairs for investment level {D}")

        GT_indices = set()
        processed_pairs = set()
        
        ## conditional routing 
        # ltn --> ltn (all)
        # ltn --> tess (all to one)
        # tess --> tess (one to one)
        # tess --> ltn (one to all)

        for u, v in routenodepairs:
            poipair = (u, v)
            if poipair in global_processed_pairs or tuple(reversed(poipair)) in global_processed_pairs:
                continue

            is_u_neighbourhood = u in all_centroids['nearest_node'].values
            is_v_neighbourhood = v in all_centroids['nearest_node'].values

            if debug:
                print(f"[{scenario}] Routing pair {u} -> {v} | Neighbourhood status: u={is_u_neighbourhood}, v={is_v_neighbourhood}")

            best_path = None
            shortest_path_length = float('inf')

            if is_u_neighbourhood and is_v_neighbourhood:
                neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
                neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
                exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_a, 'osmid']
                exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_b, 'osmid']

                if debug:
                    print(f"[{scenario}] Available exit points for u ({u}, neighbourhood {neighbourhood_a}): {list(exit_points_a)}")
                    print(f"[{scenario}] Available exit points for v ({v}, neighbourhood {neighbourhood_b}): {list(exit_points_b)}")

                processed_pairs = set()
                for ea in exit_points_a:
                    for eb in exit_points_b:
                        pair_id = tuple(sorted((ea, eb)))
                        if pair_id in processed_pairs:
                            continue
                        processed_pairs.add(pair_id)
                        try:
                            sp = nx.shortest_path(G_weighted, source=ea, target=eb, weight='length')
                            sp_length = nx.shortest_path_length(G_weighted, source=ea, target=eb, weight='length')

                            if sp_length < shortest_path_length:
                                shortest_path_length = sp_length
                                best_path = sp
                        except nx.NetworkXNoPath:
                            continue

            elif is_u_neighbourhood and not is_v_neighbourhood:
                neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
                exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']

                if debug:
                    print(f"[{scenario}] Available exit points for u ({u}, neighbourhood {neighbourhood_id}): {list(exit_points_a)}")

                for ea in exit_points_a:
                    try:
                        sp = nx.shortest_path(G_weighted, source=ea, target=v, weight='length')
                        sp_length = nx.shortest_path_length(G_weighted, source=ea, target=v, weight='length')
                        if sp_length < shortest_path_length:
                            shortest_path_length = sp_length
                            best_path = sp
                    except nx.NetworkXNoPath:
                        continue

            elif not is_u_neighbourhood and is_v_neighbourhood:
                neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
                exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']

                if debug:
                    print(f"[{scenario}] Available exit points for v ({v}, neighbourhood {neighbourhood_id}): {list(exit_points_b)}")

                for eb in exit_points_b:
                    try:
                        sp = nx.shortest_path(G_weighted, source=u, target=eb, weight='length')
                        sp_length = nx.shortest_path_length(G_weighted, source=u, target=eb, weight='length')
                        if sp_length < shortest_path_length:
                            shortest_path_length = sp_length
                            best_path = sp
                    except nx.NetworkXNoPath:
                        continue

            else:
                try:
                    sp = nx.shortest_path(G_weighted, source=u, target=v, weight='length')
                    shortest_path_length = nx.shortest_path_length(G_weighted, source=u, target=v, weight='length')
                    best_path = sp
                except nx.NetworkXNoPath:
                    if debug:
                        print(f"[{scenario}] No path found between {u} and {v}")

            if best_path:
                GT_indices.update(best_path)
                if debug:
                    print(f"[{scenario}] Shortest path from {u} to {v} (length: {shortest_path_length:.2f} m): {best_path}")
    

            else:
                if debug:
                    print(f"[{scenario}] No path found between {u} and {v}")

            global_processed_pairs.add(poipair)

        cumulative_GT_indices.update(GT_indices)

        GT = G_carall.subgraph(cumulative_GT_indices)
        for u, v, data in GT.edges(data=True):
            if 'length' in data:
                data['weight'] = data['length']

        GTs.append(GT)
        _, GT_edges = ox.graph_to_gdfs(GT)
        GTs_gdf.append(GT_edges)

        if debug:
            ax = GT_edges.to_crs(epsg=3857).plot()
            ltn_gdf.to_crs(epsg=3857).plot(ax=ax, color='red', markersize=10)
            tess_gdf.to_crs(epsg=3857).plot(ax=ax, color='green', markersize=5)
            ax.set_title(f"[{scenario}] Investment level: {D}, Number of edges: {len(GT.edges)}")
            plt.show()

    # save to plot gifs
    GTs_dict[scenario] = GTs
    GT_abstracts_dict[scenario] = GT_abstracts

    # write results
    prune_measure = "betweenness_ltn_priority"
    results = {
        "placeid": placeid,
        "prune_measure": prune_measure,
        "poi_source": params["poi_source"],
        "prune_quantiles": investment_levels_dict[(placeid, scenario)],
        "GTs": GTs,
        "GT_abstracts": GT_abstracts}

    # Save files. Files are saved to a pickle. They will be of the form "newcastle_poi_LTNs_tessellation_betweenness_weighted_current_ltn_scenario.pickle". 
    utils.write_result(results, "pickle", placeid, params["poi_source"], prune_measure, ".pickle", weighting=params["weighting"], scenario=scenario)


if gifs:
    # plot gifs
    growth_metric = "betweeness_ltn_priority"  
    for scenario in params["scenarios"]:
        if scenario == "no_ltn_scenario":
            print(f"Skipping GIF generation for scenario: {scenario}")
            continue

        ltn_points = ltn_gdfs_dict.get((placeid, scenario))
        tess_points = tess_gdfs_dict.get((placeid, scenario))
        neighbourhoods = utils.load_neighbourhoods(os.path.join(PATH["data"], placeid, scenario)) if scenario != "no_ltn_scenario" else None

        for graph_type, graph_list in [("routed", GTs_dict[scenario]), ("abstract", GT_abstracts_dict[scenario])]:
            output_file = os.path.join(PATH["videos"], placeid, scenario, f"{placeid}_{scenario}_{graph_type}_{growth_metric}_growth.gif")
            title_prefix = f"{scenario} - {graph_type.capitalize()} ({growth_metric.replace('_', ' ').capitalize()}) iteration"
            utils.plot_investment_growth_gifs(
                graph_list=graph_list,
                output_path=output_file,
                G_biketrackcarall=G_caralls[(placeid, scenario)],
                G_biketrack=G_biketracks[(placeid, scenario)],
                ltn_points=ltn_points,
                tess_points=tess_points,
                neighbourhoods=neighbourhoods,
                investment_levels=investment_levels_dict[(placeid, scenario)],
                title_prefix=title_prefix)


## Potential Demand Growth

#### Loading PCT results - Predownloaded

In [ ]:
# read in lines
lines = gpd.read_file(PATH["example-data"] +  "/lines_lsoa.gpkg")
rnet = gpd.read_file(PATH["example-data"] + "/rnet_lsoa.gpkg")
lsoa = gpd.read_file(PATH["example-data"] + "/lsoa.gpkg")
lsoa_bound = gpd.read_file(PATH["example-data"] + "/lsoa_bound.gpkg")

# clip to boundary
boundary = ox.geocode_to_gdf(placeinfo["nominatimstring"])
#lines = gpd.clip(lines, boundary) # dont clip, otherwise lines which pass temporarily outside the boundary will be removed
rnet = gpd.clip(rnet, boundary)
lsoa = gpd.clip(lsoa, boundary)
lsoa_bound = gpd.clip(lsoa_bound, boundary)
valid_lad11cds = lsoa['lad11cd'].unique()
lines = lines[lines['lad11cd1'].isin(valid_lad11cds) & lines['lad11cd2'].isin(valid_lad11cds)] # do this instead


In [ ]:
if params["methods_plotting"]:
    for scenario in params["scenarios"]:
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location = locations[(placeid, scenario)]
        location_gseries = gpd.GeoSeries([location])
        location_gdf = gpd.GeoDataFrame(geometry=location_gseries)
        location_gdf.set_crs(epsg=4326, inplace=True)
        location_gdf.plot(ax=ax, color='lightgrey', zorder=0)

        # add pct lines
        lines.plot(ax=ax, column="dutch_slc", scheme='Percentiles', alpha=0.2)
        plt.axis('off')
        plt.savefig(PATH["plots"] + placeid + "/" + scenario + "/pct_raw_demand.png", dpi=600, bbox_inches='tight')
        plt.close()

    for scenario in params["scenarios"]:
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location = locations[(placeid, scenario)]
        location_gseries = gpd.GeoSeries([location])
        location_gdf = gpd.GeoDataFrame(geometry=location_gseries)
        location_gdf.set_crs(epsg=4326, inplace=True)
        location_gdf.plot(ax=ax, color='lightgrey', zorder=0)

        # add pct lines
        lines.plot(ax=ax, column="dutch_slc", scheme='Percentiles', alpha=0.2, label=scenario)

        tess_nodes = tess_nodes_dict[(placeid, scenario)]
        tess_nodes.plot(ax=ax, color='darkorange', markersize=30, zorder=5)

        if scenario != "no_ltn_scenario":
            ltn_nodes = ltn_nodes_dict[(placeid, scenario)]
            ltn_nodes.plot(ax=ax, color='darkorange', markersize=30, zorder=4)

            greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
            greedy_gdf.to_crs(lines.crs).plot(ax=ax, color='red', linewidth=0.9, zorder=3, label=scenario)

        plt.axis('off')
        plt.savefig(PATH["plots"] + placeid + "/" + scenario + "/pct_greedy_setup.png", dpi=600, bbox_inches='tight')
        plt.close()


### PCT Demand to GT

In order to get better demand data, we can transform the desire lines from the PCT's potential demand into the ordering method of links in our greedy triangulation. This can be compared against using betweeness centraility, which up until now has been used as our proxy for demand.

PCT desire lines go from LSOA centriods, which are at a different scale to our start and end points. To deal with this, we link each lsoa to its nearest seed point. We can then aggregate the demand between seed points. 

### Aggregate demand

### k-nearest neighbours

find nearest neighbours of seed points, then weight demand by inverse distance from 

This joins demand to lines via a k-Nearest Neighbour approach.

In [ ]:
for scenario in params["scenarios"]:
    combined_nodes = combined_nodes_dict[(placeid, scenario)]
    greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]

    # double check crs
    if lsoa.crs != combined_nodes.crs:
        combined_nodes = combined_nodes.to_crs(lsoa.crs)
    # Nearest seed point for each LSOA
    join = gpd.sjoin_nearest(lsoa[['geometry']], combined_nodes[['osmid', 'geometry']], how='left')
    lsoa['nearest_seed_point'] = join['osmid']

    # Reproject to work in meters
    lsoa = lsoa.to_crs(epsg=3857)
    lines = lines.to_crs(epsg=3857)
    combined_nodes = combined_nodes.to_crs(epsg=3857)
    greedy_gdf = greedy_gdf.to_crs(epsg=3857)

    # Convert geo_code columns to strings
    lsoa['geo_code'] = lsoa['geo_code'].astype(str)
    lines['geo_code1'] = lines['geo_code1'].astype(str)
    lines['geo_code2'] = lines['geo_code2'].astype(str)

    # Compute coordinates
    lsoa['x'] = lsoa.geometry.x
    lsoa['y'] = lsoa.geometry.y
    combined_nodes['x'] = combined_nodes.geometry.x
    combined_nodes['y'] = combined_nodes.geometry.y

    # k-NN setup
    k = 3  # CHANGE 
    combined_coords = combined_nodes[['x', 'y']].values
    lsoa_coords = lsoa[['x', 'y']].values

    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(combined_coords)
    distances, indices = nbrs.kneighbors(lsoa_coords)

    # Build mapping 
    mapping = {}
    for i, row in enumerate(lsoa.itertuples()):
        lsoa_code = row.geo_code
        node_indices = indices[i]
        dists = distances[i]
        weights = 1 / (dists + 1e-6)  # Avoid division by zero
        normalised_weights = weights / weights.sum()
        node_ids = combined_nodes.iloc[node_indices]['osmid'].values
        mapping[lsoa_code] = list(zip(node_ids, normalised_weights))

    if debug:
        print(f"[{scenario}] Mapping keys (first 10):", list(mapping.keys())[:10])
        print(f"[{scenario}] Total LSOA mapping entries:", len(mapping))

    # Generate flow records from desire lines
    flow_records = []
    for idx, row in lines.iterrows():
        origin_code = row['geo_code1']
        dest_code = row['geo_code2']
        demand = row['dutch_slc']

        if debug:
            if origin_code not in mapping:
                print(f"[{scenario}] Origin code {origin_code} not in mapping")
            if dest_code not in mapping:
                print(f"[{scenario}] Destination code {dest_code} not in mapping")

        if origin_code in mapping and dest_code in mapping:
            origin_nodes = mapping[origin_code]
            dest_nodes = mapping[dest_code]
            total_weight_product = sum(w_o * w_d for (_, w_o) in origin_nodes for (_, w_d) in dest_nodes)
            for o_node, w_o in origin_nodes:
                for d_node, w_d in dest_nodes:
                    flow_share = demand * (w_o * w_d) / total_weight_product
                    node_pair = tuple(sorted((o_node, d_node)))
                    flow_records.append({'osmid_pair': node_pair, 'flow': flow_share})

    if debug:
        print(f"[{scenario}] Number of flow records:", len(flow_records))

    # Aggregate flows
    flow_df = pd.DataFrame(flow_records)
    if flow_df.empty:
        print(f"[{scenario}] No flow records were generated. Check mapping and geo_code consistency.")
    else:
        total_flow = flow_df.groupby('osmid_pair')['flow'].sum().reset_index(name='total_flow')
        if debug:
            print(f"[{scenario}] Total flow (head):\n", total_flow.head())

        greedy_gdf = greedy_gdf.copy()
        greedy_gdf['osmid_pair'] = greedy_gdf.apply(
            lambda row: tuple(sorted((int(row['start_osmid']), int(row['end_osmid'])))), axis=1
        )
        greedy_gdf = greedy_gdf.merge(total_flow, on='osmid_pair', how='left')
        # shouldn't happen, but just in case
        if 'total_flow_y' in greedy_gdf.columns and debug:
            greedy_gdf['total_flow'] = greedy_gdf['total_flow_y']
        elif 'total_flow' in greedy_gdf.columns and debug:
            pass  # already fine
        else:
            print(f"[{scenario}] Warning: total_flow column not found after merge.")
        greedy_gdf['total_flow'] = greedy_gdf['total_flow'].fillna(0)

        if debug:
            print(f"[{scenario}] Merged greedy_gdf (head):\n", greedy_gdf[['osmid_pair', 'total_flow']].head())

    # Reproject back to WGS84
    lsoa = lsoa.to_crs(epsg=4326)
    lines = lines.to_crs(epsg=4326)
    combined_nodes = combined_nodes.to_crs(epsg=4326)
    greedy_gdf = greedy_gdf.to_crs(epsg=4326)

    # Optionally store updated greedy_gdf
    greedy_gdfs_dict[(placeid, scenario)] = greedy_gdf

    if params["methods_plotting"]:
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location = locations[(placeid, scenario)]
        location_gseries = gpd.GeoSeries([location])
        location_gdf = gpd.GeoDataFrame(geometry=location_gseries)
        location_gdf = location_gdf.set_crs(epsg=4326, inplace=True)
        location_gdf.plot(ax=ax, color='lightgrey', zorder=0)

        if scenario != "no_ltn_scenario":
            ltn_nodes = ltn_nodes_dict[(placeid, scenario)]
            ltn_nodes.plot(ax=ax, color='darkorange', markersize=30, zorder=5)
        tess_nodes = tess_nodes_dict[(placeid, scenario)]
        tess_nodes.plot(ax=ax, color='darkorange', markersize=30, zorder=4)
        greedy_gdf.to_crs(lines.crs).plot(ax=ax, linewidth=1.5, zorder=3, column='total_flow', cmap='viridis', alpha=0.8)

        plt.axis('off')
        plt.savefig(PATH["plots"] + placeid + "/" + scenario + "/pct_greedy.png", dpi=600,  bbox_inches='tight')
        plt.show()


### Demand Growth - Set Connection


In [ ]:
demand_ltn_dict = {}
demand_other_dict = {}
demand_all_dict = {}

for scenario in params["scenarios"]:
    key = (placeid, scenario)

    greedy_gdf = greedy_gdfs_dict[key]
    greedy_nx = nx.Graph()

    for _, row in greedy_gdf.iterrows():
        start = row['start_osmid']
        end = row['end_osmid']
        total_flow = row['total_flow']
        sp_lts_distance = row.get('sp_lts_distance', 0)

        greedy_nx.add_edge(
            start, end,
            geometry=row['geometry'],
            total_flow=total_flow,
            sp_lts_distance=sp_lts_distance
        )

    greedy_nx_dict[key] = greedy_nx  # Save graph

    # Prepare node pairs and shortest path dicts for this scenario
    combined_pairs = list(map(tuple, combined_node_pairs_dict[key]))
    sp_all = utils.make_sp_dict(shortest_paths_all_dict[key])

    if scenario != "no_ltn_scenario":
        ltn_pairs = list(map(tuple, ltn_node_pairs_dict[key]))
        sp_ltn = utils.make_sp_dict(shortest_paths_ltn_dict[key])
        sp_other = utils.make_sp_dict(shortest_paths_other_dict[key])

    # Compute demands
    if scenario != "no_ltn_scenario":
        d_ltn = utils.get_sp_demand_weights(ltn_pairs, sp_ltn, greedy_nx)
        d_other = utils.get_sp_demand_weights(combined_pairs, sp_other, greedy_nx)
    d_all = utils.get_sp_demand_weights(combined_pairs, sp_all, greedy_nx)

    # Sort and store results
    if scenario != "no_ltn_scenario":
        demand_ltn_dict[key] = dict(sorted(d_ltn.items(), key=lambda x: x[1], reverse=True))
        demand_other_dict[key] = dict(sorted(d_other.items(), key=lambda x: x[1], reverse=True))
    demand_all_dict[key] = dict(sorted(d_all.items(), key=lambda x: x[1], reverse=True))


    # Make sure CRS is updated
    greedy_gdfs_dict[key] = greedy_gdf.to_crs(3857)

### Demand Growth - Prune Greedy Triangulation

This cell creates a growth order based on **potential demand**. Priority is given to Low Traffic Neighbouhoods, only once LTNs have been connected are any other links added.

In [ ]:
# Demand-based triangulation and routing for multiple scenarios
GTs_dict = {}
GT_abstracts_dict = {}

for scenario in params["scenarios"]:
    # Skip scenario without LTNs
    if scenario == "no_ltn_scenario":
        print(f"Skipping scenario: {scenario}, no LTNs to route")
        continue

    key = (placeid, scenario)
    previous_selected_edges = set()

    GT_abstracts = []
    GT_abstracts_gdf = []
    GTs = []
    GTs_gdf = []

    global_processed_pairs = set()
    cumulative_GT_indices = set()

    # retrieve demand and shortest-path edge lists for this scenario
    greedy_gdf = greedy_gdfs_dict[key]
    sp_ltn = utils.make_sp_edge_dict(shortest_paths_ltn_dict[key])
    sp_other = utils.make_sp_edge_dict(shortest_paths_other_dict[key])
    demand_ltn = demand_ltn_dict[key]
    demand_other = demand_other_dict[key]
    G_weighted = G_weighteds[key]
    all_centroids = all_centroids_dict[(placeid, scenario)]
    tess_gdf = tess_gdfs_dict[(placeid, scenario)]
    ltn_gdf = ltn_gdfs_dict[(placeid, scenario)]
    exit_points = exit_points_dict[(placeid, scenario)]


    for D in tqdm(investment_levels_dict[(placeid, scenario)], desc=f"Pruning GT abstract and routing on network for demand-based budget - {scenario}"):
        # abstract triangulation prioritized by demand
        GT_abstract_gdf, previous_selected_edges, connected_ltn_pairs, connected_other_pairs = utils.adjust_triangulation_to_budget_ltn_priority(
                greedy_gdf, D,
                sp_ltn, demand_ltn,
                sp_other, demand_other,
                previous_selected_edges,
                ltn_node_pairs_dict[key])

        GT_abstracts_gdf.append(GT_abstract_gdf)
        GT_abstract_nx = utils.gdf_to_nx_graph(GT_abstract_gdf)
        GT_abstracts.append(GT_abstract_nx)

        if debug:
            ax = GT_abstract_gdf.plot()
            ltn_gdf.plot(ax=ax, color='red', markersize=10)
            tess_gdf.plot(ax=ax, color='green', markersize=5)
            for idx, row in ltn_gdf.iterrows():
                ax.annotate(
                    text=str(row['osmid']),
                    xy=(row.geometry.x, row.geometry.y),
                    xytext=(3, 3),
                    textcoords="offset points",
                    fontsize=8,
                    color="red"
                )

        poipairs = list(GT_abstract_nx.edges())
        routenodepairs = [(u, v) for u, v in poipairs]

        if debug:
            print(f"[{scenario}] Routing on network for investment level: {D} with routenodepairs", routenodepairs)

        GT_indices = set()
        processed_pairs = set()

        for u, v in routenodepairs:
            poipair = (u, v)
            if poipair in global_processed_pairs or tuple(reversed(poipair)) in global_processed_pairs:
                continue

            is_u_neighbourhood = u in all_centroids['nearest_node'].values
            is_v_neighbourhood = v in all_centroids['nearest_node'].values

            shortest_path_length = float('inf')
            best_path = None

            if is_u_neighbourhood and is_v_neighbourhood:
                neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
                neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
                exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_a, 'osmid']
                exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_b, 'osmid']
                for ea in exit_points_a:
                    for eb in exit_points_b:
                        pair_id = tuple(sorted((ea, eb)))
                        if pair_id in processed_pairs:
                            continue
                        processed_pairs.add(pair_id)
                        try:
                            sp = nx.shortest_path(G_weighted, source=ea, target=eb, weight='length')
                            sp_length = nx.shortest_path_length(G_weighted, source=ea, target=eb, weight='length')
                            if sp_length < shortest_path_length:
                                shortest_path_length = sp_length
                                best_path = sp
                        except nx.NetworkXNoPath:
                            continue
                if best_path:
                    GT_indices.update(best_path)

            elif is_u_neighbourhood and not is_v_neighbourhood:
                neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
                exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']
                for ea in exit_points_a:
                    try:
                        sp = nx.shortest_path(G_weighted, source=ea, target=v, weight='length')
                        sp_length = nx.shortest_path_length(G_weighted, source=ea, target=v, weight='length')
                        if sp_length < shortest_path_length:
                            shortest_path_length = sp_length
                            best_path = sp
                    except nx.NetworkXNoPath:
                        continue
                if best_path:
                    GT_indices.update(best_path)

            elif not is_u_neighbourhood and is_v_neighbourhood:
                neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
                exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']
                for eb in exit_points_b:
                    try:
                        sp = nx.shortest_path(G_weighted, source=u, target=eb, weight='length')
                        sp_length = nx.shortest_path_length(G_weighted, source=u, target=eb, weight='length')
                        if sp_length < shortest_path_length:
                            shortest_path_length = sp_length
                            best_path = sp
                    except nx.NetworkXNoPath:
                        continue
                if best_path:
                    GT_indices.update(best_path)

            else:
                try:
                    sp = nx.shortest_path(G_weighted, source=u, target=v, weight='length')
                    GT_indices.update(sp)
                except nx.NetworkXNoPath:
                    continue

            global_processed_pairs.add(poipair)

        cumulative_GT_indices.update(GT_indices)

        GT = G_caralls[key].subgraph(cumulative_GT_indices)

        for u, v, data in GT.edges(data=True):
            if 'length' in data:
                data['weight'] = data['length']

        GTs.append(GT)
        _, GT_edges = ox.graph_to_gdfs(GT)
        GTs_gdf.append(GT_edges)

        if debug:
            GT_nodes, GT_edges = ox.graph_to_gdfs(GT)
            GT_edges = GT_edges.to_crs(epsg=3857)
            ax = GT_edges.plot()
            ltn_gdf = ltn_nodes_dict[(placeid, scenario)].to_crs(epsg=3857)
            tess_gdf = tess_nodes_dict[(placeid, scenario)].to_crs(epsg=3857)
            ltn_gdf.plot(ax=ax, color='red', markersize=10)
            tess_gdf.plot(ax=ax, color='green', markersize=5)
            ax.set_title(f"[{scenario}] Investment level: {D}, Number of edges: {len(GT.edges)}")
            plt.show()

    GTs_dict[scenario] = GTs
    GT_abstracts_dict[scenario] = GT_abstracts

    # save results for this scenario
    prune_measure = "demand_ltn_priority"
    results = {
        "placeid": placeid,
        "prune_measure": prune_measure,
        "poi_source": params["poi_source"],
        "prune_quantiles": investment_levels_dict[(placeid, scenario)],
        "GTs": GTs,
        "GT_abstracts": GT_abstracts
    }
    utils.write_result(results, "pickle", placeid, params["poi_source"], prune_measure, ".pickle", weighting=params["weighting"], scenario=scenario)

if gifs:
    # plot gifs
    growth_metric = "demand_ltn_priority"  
    for scenario in params["scenarios"]:
        if scenario == "no_ltn_scenario":
            print(f"Skipping GIF generation for scenario: {scenario}")
            continue

        ltn_points = ltn_gdfs_dict.get((placeid, scenario))
        tess_points = tess_gdfs_dict.get((placeid, scenario))
        neighbourhoods = utils.load_neighbourhoods(os.path.join(PATH["data"], placeid, scenario)) if scenario != "no_ltn_scenario" else None

        for graph_type, graph_list in [("routed", GTs_dict[scenario]), ("abstract", GT_abstracts_dict[scenario])]:
            output_file = os.path.join(PATH["videos"], placeid, scenario, f"{placeid}_{scenario}_{graph_type}_{growth_metric}_growth.gif")
            title_prefix = f"{scenario} - {graph_type.capitalize()} ({growth_metric.replace('_', ' ').capitalize()}) iteration"
            utils.plot_investment_growth_gifs(
                graph_list=graph_list,
                output_path=output_file,
                G_biketrackcarall=G_caralls[(placeid, scenario)],
                G_biketrack=G_biketracks[(placeid, scenario)],
                ltn_points=ltn_points,
                tess_points=tess_points,
                neighbourhoods=neighbourhoods,
                investment_levels=investment_levels_dict[(placeid, scenario)],
                title_prefix=title_prefix
            )



# LTNs Not Prioritised 

The previous section of code has produced bike networks where the growth between LTNs has been prioiritsed over any other link. This section produces growth plans where LTNs aren't given explict priority

## Random Growth - Prune Greedy Triangulation

This cell creates many iterations of a **random** growth order. No priority is given to Low Traffic Neighbouhoods. Many runs are required to ensure we get a true sense of what a random strategy would look like. 

**Note** - creating many runs can take some time, especially if you are running more than one scenario. Good things come to those who wait :D

In [ ]:
for scenario in params["scenarios"]:
    key = (placeid, scenario)
    greedy_gdf = greedy_gdfs_dict[key]
    G_weighted = G_weighteds[key]
    G_caralls_single = G_caralls[key]
    all_centroids = all_centroids_dict[key]
    if scenario != "no_ltn_scenario":
        exit_points = exit_points_dict[key]
        ltn_gdf = ltn_gdfs_dict[key]
    tess_gdf = tess_gdfs_dict[key]

    shuffled_edges = greedy_gdf.sample(frac=1, random_state=42)

    if scenario != "no_ltn_scenario":
        exit_nodes = set(exit_points['osmid'])
        endpoints = set(greedy_gdf['start_osmid']).union(greedy_gdf['end_osmid'])
        lookup_nodes = exit_nodes.union(endpoints)
    else:
        endpoints = set(greedy_gdf['start_osmid']).union(greedy_gdf['end_osmid'])
        lookup_nodes = endpoints

    print("Creating shortest paths lookup for", scenario)
    sp_length = {}
    sp_path = {}
    for source in lookup_nodes:
        dist_dict, path_dict = nx.single_source_dijkstra(G_weighted, source=source, weight='length')
        for target in lookup_nodes:
            if target in dist_dict:
                sp_length[(source, target)] = dist_dict[target]
                sp_path[(source, target)] = path_dict[target]

    global_cache = {'shuffled_edges': shuffled_edges, 'sp_length': sp_length, 'sp_path': sp_path}

    iterations = range(1, 101)
    if scenario != "no_ltn_scenario":
        for run_id in tqdm(iterations, desc=f"Random Growth Runs - {scenario}", unit="Run"):
            results = utils.run_random_growth(
                placeid=placeid,
                poi_source=params["poi_source"],
                investment_levels=investment_levels_dict[(placeid, scenario)],
                weighting=params["weighting"],
                greedy_gdf=greedy_gdf,
                G_caralls=G_caralls_single,
                G_weighted=G_weighted,
                all_centroids=all_centroids,
                exit_points=exit_points,
                sp_length=sp_length,
                sp_path=sp_path,
                ltn_gdf=ltn_gdf,
                tess_gdf=tess_gdf,
                debug=False
            )

            suffix = f"_run{run_id:02d}.pickle"
            utils.write_result(
                results=results,
                file_format = "pickle",
                placeid=placeid,
                poi_source=params["poi_source"],
                prune_measure="random",
                extension=suffix,
                weighting=params["weighting"],
                scenario=scenario)
    else:
        for run_id in tqdm(iterations, desc=f"Random Growth Runs - {scenario}", unit="Run"):
            results = utils.run_random_growth_no_ltn(
                placeid=placeid,
                poi_source=params["poi_source"],
                investment_levels=investment_levels_dict[(placeid, scenario)],
                weighting=params["weighting"],
                greedy_gdf=greedy_gdf,
                G_caralls=G_caralls_single,
                G_weighted=G_weighted,
                all_centroids=all_centroids,
                sp_length=sp_length,
                sp_path=sp_path,
                tess_gdf=tess_gdf,
                debug=False
            )

            suffix = f"_run{run_id:02d}.pickle"
            utils.write_result(
                results=results,
                file_format = "pickle",
                placeid=placeid,
                poi_source=params["poi_source"],
                prune_measure="random",
                extension=suffix,
                weighting=params["weighting"],
                scenario=scenario )


## Betweenness Growth - Prune Greedy Triangulation

This cell creates a growth order based on **betweeness centrailty**. No priority is given to Low Traffic Neighbouhoods.

In [ ]:
# betweenness
GTs_dict = {}
GT_abstracts_dict = {}

for scenario in params["scenarios"]:
    previous_selected_edges = set()
    GT_abstracts = []
    GT_abstracts_gdf = []
    GTs = []
    GTs_gdf = []
    global_processed_pairs = set()
    cumulative_GT_indices = set()
    G_weighted = G_weighteds[(placeid, scenario)]
    G_carall = G_caralls[(placeid, scenario)]

    greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
    shortest_paths_all = shortest_paths_all_pairs_dict[(placeid, scenario)]
    ebc_all = ebc_all_dict[(placeid, scenario)]
    ltn_gdf = ltn_gdfs_dict[(placeid, scenario)] if scenario != "no_ltn_scenario" else None
    tess_gdf = tess_gdfs_dict[(placeid, scenario)]
    all_centroids = all_centroids_dict[(placeid, scenario)]
    exit_points = exit_points_dict[(placeid, scenario)] if scenario != "no_ltn_scenario" else None

    for D in tqdm(investment_levels_dict[(placeid, scenario)], desc=f"Pruning GT abstract and routing on network for meters of investment - {scenario}"):
        # make abstract greedy triangulation graph
        GT_abstract_gdf, previous_selected_edges, connected_pairs = utils.adjust_triangulation_to_budget(
            greedy_gdf, D, shortest_paths_all, ebc_all, previous_selected_edges
        )

        remaining_edges = len(greedy_gdf) - len(previous_selected_edges)
        if debug:
            print(f"[{scenario}] Remaining edges to add: {remaining_edges}")

        GT_abstracts_gdf.append(GT_abstract_gdf)
        GT_abstract_nx = utils.gdf_to_nx_graph(GT_abstract_gdf)
        GT_abstracts.append(GT_abstract_nx)

        if debug:
            ax = GT_abstract_gdf.to_crs(epsg=3857).plot()
            if scenario != "no_ltn_scenario":
                ltn_gdf.to_crs(epsg=3857).plot(ax=ax, color='red', markersize=10)
            tess_gdf.to_crs(epsg=3857).plot(ax=ax, color='green', markersize=5)
            if scenario != "no_ltn_scenario":
                for idx, row in ltn_gdf.iterrows():
                    ax.annotate(
                        text=str(row['osmid']),
                        xy=(row.geometry.x, row.geometry.y),
                        xytext=(3, 3),
                        textcoords="offset points",
                        fontsize=8,
                        color="red"
                    )

        poipairs = list(GT_abstract_nx.edges())
        routenodepairs = [(u, v) for u, v in poipairs]


        if debug:
            print(f"[{scenario}] Routing on network for investment level: {D} with routenodepairs", routenodepairs)

        GT_indices = set()
        processed_pairs = set()

        for u, v in routenodepairs:
            poipair = (u, v)
            if poipair in global_processed_pairs or tuple(reversed(poipair)) in global_processed_pairs:
                continue

            is_u_neighbourhood = u in all_centroids['nearest_node'].values
            is_v_neighbourhood = v in all_centroids['nearest_node'].values

            if is_u_neighbourhood and is_v_neighbourhood:
                neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
                neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
                exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_a, 'osmid']
                exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_b, 'osmid']
                shortest_path_length, best_path = float('inf'), None
                for ea in exit_points_a:
                    for eb in exit_points_b:
                        pair_id = tuple(sorted((ea, eb)))
                        if pair_id in processed_pairs:
                            continue
                        processed_pairs.add(pair_id)
                        try:
                            sp = nx.shortest_path(G_weighted, source=ea, target=eb, weight='length')
                            sp_length = nx.shortest_path_length(G_weighted, source=ea, target=eb, weight='length')
                            if sp_length < shortest_path_length:
                                shortest_path_length, best_path = sp_length, sp
                        except nx.NetworkXNoPath:
                            continue
                if best_path:
                    GT_indices.update(best_path)

            elif is_u_neighbourhood and not is_v_neighbourhood:
                neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
                exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']
                shortest_path_length, best_path = float('inf'), None
                for ea in exit_points_a:
                    try:
                        sp = nx.shortest_path(G_weighted, source=ea, target=v, weight='length')
                        sp_length = nx.shortest_path_length(G_weighted, source=ea, target=v, weight='length')
                        if sp_length < shortest_path_length:
                            shortest_path_length, best_path = sp_length, sp
                    except nx.NetworkXNoPath:
                        continue
                if best_path:
                    GT_indices.update(best_path)

            elif not is_u_neighbourhood and is_v_neighbourhood:
                neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
                exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']
                shortest_path_length, best_path = float('inf'), None
                for eb in exit_points_b:
                    try:
                        sp = nx.shortest_path(G_weighted, source=u, target=eb, weight='length')
                        sp_length = nx.shortest_path_length(G_weighted, source=u, target=eb, weight='length')
                        if sp_length < shortest_path_length:
                            shortest_path_length, best_path = sp_length, sp
                    except nx.NetworkXNoPath:
                        continue
                if best_path:
                    GT_indices.update(best_path)

            else:
                try:
                    sp = nx.shortest_path(G_weighted, source=u, target=v, weight='length')
                    GT_indices.update(sp)
                except nx.NetworkXNoPath:
                    continue

            global_processed_pairs.add(poipair)
        
    
        cumulative_GT_indices.update(GT_indices)
     

        GT = G_carall.subgraph(cumulative_GT_indices)
        for u, v, data in GT.edges(data=True):
            if 'length' in data:
                data['weight'] = data['length']


        GTs.append(GT)
        GT_nodes, GT_edges = ox.graph_to_gdfs(GT)
        GTs_gdf.append(GT_edges)

        if debug:
            ax = GT_edges.to_crs(epsg=3857).plot()
            if scenario != "no_ltn_scenario":
                ltn_gdfs_dict[(placeid, scenario)].to_crs(epsg=3857).plot(ax=ax, color='red', markersize=10)
            tess_gdfs_dict[(placeid, scenario)].to_crs(epsg=3857).plot(ax=ax, color='green', markersize=5)
            ax.set_title(f"[{scenario}] Investment level: {D}, Number of edges: {len(GT.edges)}")
            plt.show()


    # save to plot gifs
    GTs_dict[scenario] = GTs
    GT_abstracts_dict[scenario] = GT_abstracts

    prune_measure = "betweenness"
    results = {
        "placeid": placeid,
        "prune_measure": prune_measure,
        "poi_source": params["poi_source"],
        "prune_quantiles": investment_levels_dict[(placeid, scenario)],
        "GTs": GTs,
        "GT_abstracts": GT_abstracts
    }
    utils.write_result(results, "pickle", placeid, params["poi_source"], prune_measure, ".pickle", weighting=params["weighting"], scenario=scenario)


if gifs:
    growth_metric = "betweeness"  
    for scenario in params["scenarios"]:
        ltn_points = ltn_gdfs_dict.get((placeid, scenario))
        tess_points = tess_gdfs_dict.get((placeid, scenario))
        neighbourhoods = utils.load_neighbourhoods(os.path.join(PATH["data"], placeid, scenario)) if scenario != "no_ltn_scenario" else None

        for graph_type, graph_list in [("routed", GTs_dict[scenario]), ("abstract", GT_abstracts_dict[scenario])]:
            output_file = os.path.join(PATH["videos"], placeid, scenario, f"{placeid}_{scenario}_{graph_type}_{growth_metric}_growth.gif")
            title_prefix = f"{scenario} - {graph_type.capitalize()} ({growth_metric.replace('_', ' ').capitalize()}) iteration"
            utils.plot_investment_growth_gifs(
                graph_list=graph_list,
                output_path=output_file,
                G_biketrackcarall=G_caralls[(placeid, scenario)],
                G_biketrack=G_biketracks[(placeid, scenario)],
                ltn_points=ltn_points,
                tess_points=tess_points,
                neighbourhoods=neighbourhoods,
                investment_levels=investment_levels_dict[(placeid, scenario)],
                title_prefix=title_prefix
            )





In [ ]:
pairs_ltn = set(debug_routing_pairs_dict.get(("ltn_priority", scenario), []))
pairs_noprio = set(debug_routing_pairs_dict.get(("no_priority", scenario), []))

extra_in_ltn = pairs_ltn - pairs_noprio
extra_in_noprio = pairs_noprio - pairs_ltn

print(f"\nPairs routed in LTN-priority but not in non-priority: {len(extra_in_ltn)}")
print(list(extra_in_ltn)[:10])

print(f"\nPairs routed in non-priority but not in LTN-priority: {len(extra_in_noprio)}")
print(list(extra_in_noprio)[:10])


In [ ]:
ltn_extra

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

extra_edges = ltn_extra.reset_index()
gdf = gpd.GeoDataFrame(extra_edges, geometry="geometry", crs=gt_edges_ltn.crs)

gdf.plot(figsize=(8, 8), linewidth=2, color="red")
plt.title("Edges in LTN-priority but not in non-priority")
plt.show()


In [ ]:
# Compare the final routed graphs
scenario = "current_ltn_scenario"
gt_edges_ltn = GT_edges_debug_dict.get(("ltn_priority", scenario))
gt_edges_noprio = GT_edges_debug_dict.get(("no_priority", scenario))

if gt_edges_ltn is not None and gt_edges_noprio is not None:
    print("\n[Comparison] Edges in LTN-priority but not in non-priority:")
    ltn_extra = gt_edges_ltn[~gt_edges_ltn.index.isin(gt_edges_noprio.index)]
    print(ltn_extra.reset_index()[["u", "v", "length"]])


    print("\n[Comparison] Edges in non-priority but not in LTN-priority:")
    noprio_extra = gt_edges_noprio[~gt_edges_noprio.index.isin(gt_edges_ltn.index)]
    print(noprio_extra.reset_index()[["u", "v", "length"]])

else:
    print("Could not compare: one or both GTs missing.")


## Demand Growth - Prune Greedy Triangulation

This cell creates a growth order based on **potential demand**. No priority is given to Low Traffic Neighbouhoods.

In [ ]:
# demand
GTs_dict = {}
GT_abstracts_dict = {}

for scenario in params["scenarios"]:
    previous_selected_edges = set()
    GT_abstracts = []
    GT_abstracts_gdf = []
    GTs = []
    GTs_gdf = []
    global_processed_pairs = set()
    cumulative_GT_indices = set()
    G_weighted = G_weighteds[(placeid, scenario)]
    G_carall = G_caralls[(placeid, scenario)]

    greedy_gdf = greedy_gdfs_dict[(placeid, scenario)]
    shortest_paths_all = shortest_paths_all_pairs_dict[(placeid, scenario)]
    demand_all = demand_all_dict[(placeid, scenario)]
    ltn_gdf = ltn_gdfs_dict[(placeid, scenario)] if scenario != "no_ltn_scenario" else None
    tess_gdf = tess_gdfs_dict[(placeid, scenario)]
    all_centroids = all_centroids_dict[(placeid, scenario)] 
    exit_points = exit_points_dict[(placeid, scenario)] if scenario != "no_ltn_scenario" else None

    for D in tqdm(investment_levels_dict[(placeid, scenario)], desc=f"Pruning GT abstract and routing on network for meters of investment - {scenario}"):
        # make abstract greedy triangulation graph
        GT_abstract_gdf, previous_selected_edges, connected_pairs = utils.adjust_triangulation_to_budget(
            greedy_gdf, D, shortest_paths_all, demand_all, previous_selected_edges
        )

        remaining_edges = len(greedy_gdf) - len(previous_selected_edges)
        if debug:
            print(f"[{scenario}] Remaining edges to add: {remaining_edges}")
        
        GT_abstracts_gdf.append(GT_abstract_gdf)
        GT_abstract_nx = utils.gdf_to_nx_graph(GT_abstract_gdf)
        GT_abstracts.append(GT_abstract_nx)

        if debug:
            ax = GT_abstract_gdf.to_crs(epsg=3857).plot()
            if scenario != "no_ltn_scenario":
                ltn_gdf.to_crs(epsg=3857).plot(ax=ax, color='red', markersize=10)
            tess_gdf.to_crs(epsg=3857).plot(ax=ax, color='green', markersize=5)
            if scenario != "no_ltn_scenario":
                for idx, row in ltn_gdf.iterrows():
                    ax.annotate(
                        text=str(row['osmid']),
                        xy=(row.geometry.x, row.geometry.y),
                        xytext=(3, 3),
                        textcoords="offset points",
                        fontsize=8,
                        color="red"
                    )
        
        poipairs = list(GT_abstract_nx.edges())
        routenodepairs = [(u, v) for u, v in poipairs]

        if debug:
            print(f"[{scenario}] Routing on network for investment level: {D} with routenodepairs", routenodepairs)

        GT_indices = set()
        processed_pairs = set()

        for u, v in routenodepairs:
            poipair = (u, v)
            if poipair in global_processed_pairs or tuple(reversed(poipair)) in global_processed_pairs:
                continue
            
            is_u_neighbourhood = u in all_centroids['nearest_node'].values
            is_v_neighbourhood = v in all_centroids['nearest_node'].values

            if is_u_neighbourhood and is_v_neighbourhood:
                neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
                neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
                exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_a, 'osmid']
                exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_b, 'osmid']
                shortest_path_length, best_path = float('inf'), None
                for ea in exit_points_a:
                    for eb in exit_points_b:
                        pair_id = tuple(sorted((ea, eb)))
                        if pair_id in processed_pairs:
                            continue
                        processed_pairs.add(pair_id)
                        try:
                            sp = nx.shortest_path(G_weighted, source=ea, target=eb, weight='length')
                            sp_length = nx.shortest_path_length(G_weighted, source=ea, target=eb, weight='length')
                            if sp_length < shortest_path_length:
                                shortest_path_length, best_path = sp_length, sp
                        except nx.NetworkXNoPath:
                            continue
                if best_path:
                    GT_indices.update(best_path)

            elif is_u_neighbourhood and not is_v_neighbourhood:
                neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
                exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']
                shortest_path_length, best_path = float('inf'), None
                for ea in exit_points_a:
                    try:
                        sp = nx.shortest_path(G_weighted, source=ea, target=v, weight='length')
                        sp_length = nx.shortest_path_length(G_weighted, source=ea, target=v, weight='length')
                        if sp_length < shortest_path_length:
                            shortest_path_length, best_path = sp_length, sp
                    except nx.NetworkXNoPath:
                        continue
                if best_path:
                    GT_indices.update(best_path)

            elif not is_u_neighbourhood and is_v_neighbourhood:
                neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
                exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']
                shortest_path_length, best_path = float('inf'), None
                for eb in exit_points_b:
                    try:
                        sp = nx.shortest_path(G_weighted, source=u, target=eb, weight='length')
                        sp_length = nx.shortest_path_length(G_weighted, source=u, target=eb, weight='length')
                        if sp_length < shortest_path_length:
                            shortest_path_length, best_path = sp_length, sp
                    except nx.NetworkXNoPath:
                        continue
                if best_path:
                    GT_indices.update(best_path)

            elif not is_u_neighbourhood and not is_v_neighbourhood:
                try:
                    sp = nx.shortest_path(G_weighted, source=u, target=v, weight='length')
                    GT_indices.update(sp)
                except nx.NetworkXNoPath:
                    continue

            global_processed_pairs.add(poipair)

        cumulative_GT_indices.update(GT_indices)

        GT = G_carall.subgraph(cumulative_GT_indices)
        for u, v, data in GT.edges(data=True):
            if 'length' in data:
                data['weight'] = data['length']

        GTs.append(GT)
        GT_nodes, GT_edges = ox.graph_to_gdfs(GT)
        GTs_gdf.append(GT_edges)

        if debug:
            ax = GT_edges.to_crs(epsg=3857).plot()
            if scenario != "no_ltn_scenario":
                ltn_gdfs_dict[(placeid, scenario)].to_crs(epsg=3857).plot(ax=ax, color='red', markersize=10)
            tess_gdfs_dict[(placeid, scenario)].to_crs(epsg=3857).plot(ax=ax, color='green', markersize=5)
            ax.set_title(f"[{scenario}] Investment level: {D}, Number of edges: {len(GT.edges)}")
            plt.show()

    # save to plot gifs
    GTs_dict[scenario] = GTs
    GT_abstracts_dict[scenario] = GT_abstracts
    
    prune_measure = "demand"
    # save results
    results = {
        "placeid": placeid,
        "prune_measure": prune_measure,
        "poi_source": params["poi_source"],
        "prune_quantiles": investment_levels_dict[(placeid, scenario)],
        "GTs": GTs,
        "GT_abstracts": GT_abstracts
    }
    utils.write_result(
        results,
        "pickle",
        placeid,
        params["poi_source"],
        prune_measure,
        ".pickle",
        weighting=params["weighting"],
        scenario=scenario
    )

if gifs:
    growth_metric = "demand"  
    for scenario in params["scenarios"]:
        ltn_points = ltn_gdfs_dict.get((placeid, scenario))
        tess_points = tess_gdfs_dict.get((placeid, scenario))
        neighbourhoods = utils.load_neighbourhoods(os.path.join(PATH["data"], placeid, scenario)) if scenario != "no_ltn_scenario" else None
        for graph_type, graph_list in [("routed", GTs_dict[scenario]), ("abstract", GT_abstracts_dict[scenario])]:
            output_file = os.path.join(PATH["videos"], placeid, scenario, f"{placeid}_{scenario}_{graph_type}_{growth_metric}_growth.gif")
            title_prefix = f"{scenario} - {graph_type.capitalize()} ({growth_metric.replace('_', ' ').capitalize()}) iteration"
            utils.plot_investment_growth_gifs(
                graph_list=graph_list,
                output_path=output_file,
                G_biketrackcarall=G_caralls[(placeid, scenario)],
                G_biketrack=G_biketracks[(placeid, scenario)],
                ltn_points=ltn_points,
                tess_points=tess_points,
                neighbourhoods=neighbourhoods,
                investment_levels=investment_levels_dict[(placeid, scenario)],
                title_prefix=title_prefix
            )


In [ ]:
print(f"Finished processing placeid: {placeid} with scenarios: {params['scenarios']}. You can now move to running script 05")

# Testing - You don't need to run any of the code below here!

The important code above is finished :-)

#### Expermient with determining k in pct aggregation

The number of neighbours used in in the k-nn has an impact on how accurate assigning demand will be. Currently testing adaptive demand and using an elbow method

In [ ]:
# from sklearn.neighbors import NearestNeighbors

# k_test = 5  # Test with different values
# nbrs = NearestNeighbors(n_neighbors=k_test).fit(lsoa[['x', 'y']])
# distances, _ = nbrs.kneighbors(lsoa[['x', 'y']])

# # Sort distances to the k-th neighbor
# sorted_distances = np.sort(distances[:, k_test-1])

# # Plot the sorted k-distances
# plt.plot(sorted_distances)
# plt.xlabel("Points Sorted by Distance")
# plt.ylabel(f"Distance to {k_test}-th Nearest Neighbor")
# plt.title("k-Distance Graph")
# plt.show()

In [ ]:
# from sklearn.neighbors import NearestNeighbors

# # Compute average distance to nearest 5 neighbors
# nbrs = NearestNeighbors(n_neighbors=5).fit(lsoa[['x', 'y']])
# distances, _ = nbrs.kneighbors(lsoa[['x', 'y']])
# avg_dist = distances.mean(axis=1)

# # Set adaptive k: more neighbors in sparse regions, fewer in dense regions
# adaptive_k = np.round(1 + (avg_dist / avg_dist.max()) * 10).astype(int)

# print("Adaptive k values (first 100):", adaptive_k[:100])


In [ ]:
# greedy_gdf.explore(column='total_flow', cmap='viridis', legend=True, legend_name='Dutch SLC')

#### Produce a network using routed PCT dutch scenario

In [ ]:

# # Sort the GeoDataFrame by 'dutch_slc' descending - most potential first
# sorted_rnet = rnet.sort_values('dutch_slc', ascending=False)
# cumulative_lengths = sorted_rnet['length_m'].cumsum().values

# G_pct_gdfs = []

# # Track the maximum index reached to avoid reprocessing
# max_idx = 0

# for D in investment_levels:
#     # Find the furthest index where cumulative length <= D
#     idx = np.searchsorted(cumulative_lengths[max_idx:], D, side='right') + max_idx
#     idx = min(idx, len(sorted_rnet))  # Ensure we don't exceed the dataframe
    
#     # Select edges up to this index
#     subset = sorted_rnet.iloc[:idx]
#     G_pct_gdfs.append(subset)
    
#     # Update max_idx to reflect edges already included
#     max_idx = idx



### General Testing

Attempt at getting population into buildings...

In [ ]:
# import ukcensusapi.Nomisweb as census_api

# os.environ["NOMIS_API_KEY"] = "0x98f2ffbbe685f2623fa5c201d4ff86a8c9c46dee"

# api = census_api.Nomisweb(cache_dir = PATH["data"]+ "/" + placeid)

# # Define the dataset and geography
# dataset_id = "NM_2010_1"  # Ensure this is the correct dataset ID
# geography = "LSOA11"      # 2021 LSOA geography type
# measures = ["OBS_VALUE"]  # Population measure

# # List of LSOA codes you're interested in
# lsoa_codes = lsoa_bound["geo_code"].tolist()  # Assuming lsoa_bound is your GeoDataFrame

# # Fetch data for the specified LSOA codes
# population_data = api.get_data(
#     dataset_id,
#     {"geography": geography, "measures": measures, "geography_code": lsoa_codes, "date": "latest"}
# )

# # Display the fetched data
# print(population_data.head())

In [ ]:
# investment_levels = [5503.54106,
#  369233.16903,
#  372964.797]

In [ ]:
# ## many combinations
# random_edges = pd.Series(False, index=greedy_gdf.index)  
# distance = 0.0 # needed for budget

# Random_GT_abstracts = []
# Random_GTs = []

# # reset global variables
# global_processed_pairs_random = set()
# cumulative_GT_indices_random = set()


# for D in tqdm(investment_levels, desc="Pruning GT abstract randomly and routing on network for meters of investment"):
#     # make abstract greedy triangulation graph
#     # Calculate remaining budget for new edges
#     remaining_budget = D - distance
    
#     if remaining_budget > 0:
#         unselected_edges = greedy_gdf[~random_edges]  # Get edges not yet selected
#         if not unselected_edges.empty:
#             shuffled_edges = unselected_edges.sample(frac=1) # Shuffle unselected edges to randomize selection
#             cumulative_distances = shuffled_edges['distance'].cumsum()
#             within_budget = cumulative_distances <= remaining_budget  # Find edges that fit within the remaining budget
#             new_indices = shuffled_edges[within_budget].index   # Get indices of edges to add
#             random_edges[new_indices] = True
#             remaining_edges = (~random_edges).sum()
#             print(f"Remaining edges to add: {remaining_edges}")
#             if within_budget.any():
#                 distance += cumulative_distances[within_budget].iloc[-1]
#     # save edges
#     GT_abstract_gdf = greedy_gdf[random_edges].copy()
#     GT_abstract_nx = gdf_to_nx_graph(GT_abstract_gdf)
#     Random_GT_abstracts.append(GT_abstract_nx)
#     poipairs = list(GT_abstract_nx.edges()) # store currently connected points for routing

#     if debug:
#         ax = GT_abstract_gdf.plot()
#         ltn_gdf.plot(ax=ax, color='red', markersize=10)
#         tess_gdf.plot(ax=ax, color='green', markersize=5)
#         for idx, row in ltn_gdf.iterrows():
#             ax.annotate(
#                 text=str(row['osmid']),  # Use index or another column for labeling
#                 xy=(row.geometry.x, row.geometry.y),  # Get the coordinates of the point
#                 xytext=(3, 3),  # Offset for better readability
#                 textcoords="offset points",
#                 fontsize=8,  
#                 color="red"
#         )
        
    
#     routenodepairs = [(u, v) for u, v in poipairs]
   

#     if debug:
#         print(f"Routing on network for investment level: {D} with routdenodepairs", routenodepairs)
    
    
#     ## conditional routing 
#     # ltn --> ltn (all)
#     # ltn --> tess (all to one)
#     # tess --> tess (one to one)
#     # tess --> ltn (one to all)

#     GT_indices = set()
#     processed_pairs = set()

#     for u, v in routenodepairs:
#         poipair = (u, v)
#         if poipair in global_processed_pairs_random or tuple(reversed(poipair)) in global_processed_pairs_random:
#             continue
        
#         # Determine if nodes are neighbourhood or tessellation
#         is_u_neighbourhood = u in all_centroids['nearest_node'].values
#         is_v_neighbourhood = v in all_centroids['nearest_node'].values
        
#         if is_u_neighbourhood and is_v_neighbourhood:
#             # Both nodes are neighbourhoods
#             neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
#             neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
            
#             exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_a, 'osmid']
#             exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_b, 'osmid']
            
#             shortest_path_length, best_path = float('inf'), None
#             for ea in exit_points_a:
#                 for eb in exit_points_b:
#                     pair_id = tuple(sorted((ea, eb)))
#                     if pair_id in processed_pairs:
#                         continue
#                     processed_pairs.add(pair_id)
                    
#                     try:
#                         sp = nx.shortest_path(G_weighted, source=ea, target=eb, weight='length')
#                         sp_length = nx.shortest_path_length(G_weighted, source=ea, target=eb, weight='length')
#                         if sp_length < shortest_path_length:
#                             shortest_path_length, best_path = sp_length, sp
#                     except nx.NetworkXNoPath:
#                         continue
            
#             if best_path:
#                 GT_indices.update(best_path)
#                 if debug:
#                     print("Routed between:", u, v,"on path: ", best_path)

#         elif is_u_neighbourhood and not is_v_neighbourhood:
#             # Neighbourhood to Tessellation
#             neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
#             exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']

#             shortest_path_length, best_path = float('inf'), None
#             for ea in exit_points_a:
#                 try:
#                     sp = nx.shortest_path(G_weighted, source=ea, target=v, weight='length')
#                     sp_length = nx.shortest_path_length(G_weighted, source=ea, target=v, weight='length')
#                     if sp_length < shortest_path_length:
#                         shortest_path_length, best_path = sp_length, sp
#                 except nx.NetworkXNoPath:
#                     continue
            
#             if best_path:
#                 GT_indices.update(best_path)
#                 if debug:
#                     print("Routed between:", u, v,"with a path length of", shortest_path_length , "on path: ", best_path)
#                     print("The exit points were:", exit_points_a)

#         elif not is_u_neighbourhood and is_v_neighbourhood:
#             # Tessellation to Neighbourhood
#             neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
#             exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']

#             shortest_path_length, best_path = float('inf'), None
#             for eb in exit_points_b:
#                 try:
#                     sp = nx.shortest_path(G_weighted, source=u, target=eb, weight='length')
#                     sp_length = nx.shortest_path_length(G_weighted, source=u, target=eb, weight='length')
#                     if sp_length < shortest_path_length:
#                         shortest_path_length, best_path = sp_length, sp
#                 except nx.NetworkXNoPath:
#                     continue
            
#             if best_path:
#                 GT_indices.update(best_path)
#                 if debug:
#                     print("Routed between:", u, v,"on path: ", best_path)

#         elif not is_u_neighbourhood and not is_v_neighbourhood:
#             # Tessellation to Tessellation
#             try:
#                 sp = nx.shortest_path(G_weighted, source=u, target=v, weight='length')
#                 GT_indices.update(sp)
#             except nx.NetworkXNoPath:
#                 continue

#         else:
#             print("This should never happen.")

#         global_processed_pairs.add(poipair) # track pairs that we've already routed between
    
#     cumulative_GT_indices_random.update(GT_indices) # add routes we've already calculated previously to current GT

#     # Generate subgraph for selected routes
#     GT = G_caralls[placeid].subgraph(cumulative_GT_indices_random)
#     #deweight_edges(GT, tag_lts)
#     # ensure weight attibute is stored
#     for u, v, data in GT.edges(data=True):
#         if 'length' in data:
#             data['weight'] = data['length']
            
#     Random_GTs.append(GT)
        
    

#     if debug:
#         GT_nodes, GT_edges = ox.graph_to_gdfs(GT)
#         GT_edges = GT_edges.to_crs(epsg=3857)
#         ax = GT_edges.plot()
#         ltn_gdf.plot(ax=ax, color='red', markersize=10)
#         tess_gdf.plot(ax=ax, color='green', markersize=5)
#         ax.set_title(f"Investment level: {D}, Number of edges: {len(GT.edges)}")

#     # write results
#     #results = {"placeid": placeid, "prune_measure": random, "poi_source": poi_source, "prune_quantiles": investment_levels, "GTs": Random_GTs_igraph, "GT_abstracts": Random_GT_abstracts_igraph}
#     #write_result(results, "pickle", placeid, poi_source, random, ".pickle", weighting=weighting)

        


# final_remaining = (~random_edges).sum()
# print(f"Final remaining edges: {final_remaining}")


In [ ]:
# %run -i functions.py

In [ ]:
# len(previous_selected_edges)

In [ ]:
# def get_unused_edges(gdf, selected_edges):
#     def normalize_edge(row):
#         return tuple(sorted((row['start_osmid'], row['end_osmid'])))
#     return gdf[~gdf.apply(lambda row: normalize_edge(row) in selected_edges, axis=1)]

# adjusted_gdf, selected_edges, _, _ = adjust_triangulation_to_budget_ltn_priority(greedy_gdf, D, shortest_paths_ltn, ebc_ltn, shortest_paths_other, ebc_other, previous_selected_edges, ltn_node_pairs)
# remaining_edges = len(greedy_gdf) - len(previous_selected_edges)
# unused_edges_gdf = get_unused_edges(greedy_gdf, selected_edges)

# # Plotting the results
# ax = greedy_gdf.plot(color='lightgrey', linewidth=1, alpha=0.5)  # background context
# unused_edges_gdf.plot(ax=ax, color='red', linewidth=2)

# unused_edges_gdf.explore()

In [ ]:
# ebc_ltn.type

In [ ]:
# def adjust_triangulation_to_budget_ltn_priority(triangulation_gdf, D, shortest_paths_ltn, ebc_ltn, shortest_paths_other, ebc_other, previous_selected_edges=None, ltn_node_pairs=None):
#     """
#     Adjust a given triangulation to fit within the specified budget D,
#     ensuring that previously selected edges are always included.
#     Only after all ltns are connected do we move to include the growth of other areas.
#     """
#     # make a graph
#     G = nx.Graph()
#     for _, row in triangulation_gdf.iterrows():
#         G.add_edge(
#             row['start_osmid'],
#             row['end_osmid'],
#             geometry=row['geometry'],
#             distance=row['distance']
#         )

#     total_length = 0
#     selected_edges = set(tuple(sorted(edge)) for edge in (previous_selected_edges or [])) # use tuple to ensure we don't double count edges

#     # Include previously selected edges so that we aren't starting from stratch each loop through
#     for u, v in selected_edges:
#         if G.has_edge(u, v):
#             total_length += G[u][v]['distance']

#     # Track the ltns which are connected
#     connected_ltn_pairs = set()

#     # Track all other connected pairs
#     connected_other_pairs = set()

#     # Prune for ltn node pairs first
#     for (node1, node2), centrality in ebc_ltn.items():
#         if node1 in G.nodes and node2 in G.nodes:
#             edges = shortest_paths_ltn.get((node1, node2), [])
#             if edges:  # If a valid path exists
#                 # Calculate new edges and their length
#                 new_edges = [tuple(sorted((u, v))) for u, v in edges if tuple(sorted((u, v))) not in selected_edges]
#                 new_length = sum(G[min(u, v)][max(u, v)]['distance'] for u, v in new_edges)
#                 # Check if adding this path exceeds the budget D
#                 if total_length + new_length > D:
#                     continue
#                 # Add the edges to selected_edges
#                 selected_edges.update(new_edges)
#                 total_length += new_length
#                 connected_ltn_pairs.add((node1, node2))

    
#     # Check if all ltn node pairs are connected
#     if set(ltn_node_pairs).issubset(connected_ltn_pairs):
#         # Now move to all other connections (ltn to tess, tess to tess, tess to ltn etc)
#         for (node1, node2), centrality in ebc_other.items():
#             if node1 in G.nodes and node2 in G.nodes:
#                 edges = shortest_paths_other.get((node1, node2), [])
#                 if edges:  # If a valid path exists
#                     new_edges = [tuple(sorted((u, v))) for u, v in edges if tuple(sorted((u, v))) not in selected_edges]
#                     new_length = sum(G[min(u, v)][max(u, v)]['distance'] for u, v in new_edges)
#                     # Check if adding this path exceeds the budget D
#                     if total_length + new_length > D:
#                         continue
#                     # Add the edges to selected_edges
#                     selected_edges.update(new_edges)
#                     total_length += new_length
#                     connected_other_pairs.add((node1, node2))
#     # missing_pairs = [pair for pair in ltn_node_pairs if pair not in connected_ltn_pairs]


#     # edges which aren't in a shortest path won't have been selected
#     # we will add these last, as they are the least important
#     unused_edges = []
#     for _, row in triangulation_gdf.iterrows():
#         e = tuple(sorted((row['start_osmid'], row['end_osmid'])))
#         dist = row['distance']
#         if e not in selected_edges:
#             unused_edges.append((dist, e))
#     unused_edges.sort(key=lambda x: x[0])
#     for dist, edge in unused_edges:
#         if total_length + dist <= D:
#             selected_edges.add(edge)
#             total_length += dist
#         else:
#             break

#     # Build the adjusted GeoDataFrame
#     lines = []
#     distances = []
#     start_osmids = []
#     end_osmids = []


#     for u, v in selected_edges:
#         lines.append(G[u][v]['geometry'])
#         distances.append(G[u][v]['distance'])
#         start_osmids.append(u)
#         end_osmids.append(v)

#     adjusted_gdf = gpd.GeoDataFrame({
#         'geometry': lines,
#         'start_osmid': start_osmids,
#         'end_osmid': end_osmids,
#         'distance': distances,
#     }, crs=triangulation_gdf.crs)

#     return adjusted_gdf, selected_edges, connected_ltn_pairs, connected_other_pairs






In [ ]:
# ############# PLOT EVERY GT_ABSTRACT  if debug #############


# ## betwenness
# ## many combinations
# previous_selected_edges = set()

# GT_abstracts = []
# GTs = []

# global_processed_pairs = set()
# cumulative_GT_indices = set()


# for D in tqdm(investment_levels, desc="Pruning GT abstract and routing on network for meters of investment"):
#     # make abstract greedy triangulation graph
#     GT_abstract_gdf, previous_selected_edges, connected_ltn_pairs, connected_other_pairs = adjust_triangulation_to_budget_ltn_priority(greedy_gdf, D, shortest_paths_ltn, ebc_ltn, shortest_paths_other, ebc_other, previous_selected_edges, ltn_node_pairs)
#     remaining_edges = len(greedy_gdf) - len(previous_selected_edges)
#     if debug:
#         print(f"Remaining edges to add: {remaining_edges}")
    
#     GT_abstract_nx = gdf_to_nx_graph(GT_abstract_gdf)
#     GT_abstracts.append(GT_abstract_nx)
#     if debug:
#         # Plot GT_abstract_gdf with a title indicating the number of meters used to invest at that level
#         ax = GT_abstract_gdf.plot()
#         ax.set_title(f"Investment level: {D} meters")
#         ax.set_title(f"Investment level: {D} meters, Number of edges: {len(GT_abstract_gdf)}")
#         plt.show()
    


In [ ]:
# Test = GT_abstracts[2]
# Test.edges(data=True)

In [ ]:
# # List to store total lengths of each graph in GT_abstracts
# total_lengths_abstracts = []

# for G in GT_abstracts:
#     # Calculate total edge length of the graph
#     lengths = nx.get_edge_attributes(G, 'weight')
#     total_length = sum(lengths.values())
#     total_lengths_abstracts.append(total_length)

# # Create a line plot
# plt.figure(figsize=(10, 6))
# plt.plot(total_lengths_abstracts, marker='o', linestyle='-')
# plt.title('Total Length of Abstract Graphs')
# plt.xlabel('Graph Index in GT_abstracts')
# plt.ylabel('Total Length (meters)')
# plt.grid(True)
# plt.show()

In [ ]:
# # set investment max to a fully connected bicycle network, and interval to 1% incremental steps
# points = np.linspace(0, total_distance, 101)
# # Drop 0 as we don't need to consider the empty network
# investment_levels = points[1:].tolist()

In [ ]:
# test = list(GT_abstract_nx.edges())
# test

In [ ]:
# ## betwenness
# ## many combinations
# previous_selected_edges = set()

# GT_abstracts = []
# GT_abstracts_gdf = []
# GTs = []
# GTs_gdf = []



# global_processed_pairs = set()
# cumulative_GT_indices = set()


# for D in tqdm(investment_levels, desc="Pruning GT abstract and routing on network for meters of investment"):
#     # make abstract greedy triangulation graph
#     GT_abstract_gdf, previous_selected_edges, connected_ltn_pairs, connected_other_pairs = adjust_triangulation_to_budget_ltn_priority(greedy_gdf, D, shortest_paths_ltn, ebc_ltn, shortest_paths_other, ebc_other, previous_selected_edges, ltn_node_pairs)
#     remaining_edges = len(greedy_gdf) - len(previous_selected_edges)
#     if debug:
#         print(f"Remaining edges to add: {remaining_edges}")
    
#     GT_abstracts_gdf.append(GT_abstract_gdf)
#     GT_abstract_nx = gdf_to_nx_graph(GT_abstract_gdf)
#     GT_abstracts.append(GT_abstract_nx)

#     if debug:
#         ax = GT_abstract_gdf.plot()
#         ltn_gdf.plot(ax=ax, color='red', markersize=10)
#         tess_gdf.plot(ax=ax, color='green', markersize=5)
#         for idx, row in ltn_gdf.iterrows():
#             ax.annotate(
#                 text=str(row['osmid']),  # Use index or another column for labeling
#                 xy=(row.geometry.x, row.geometry.y),  # Get the coordinates of the point
#                 xytext=(3, 3),  # Offset for better readability
#                 textcoords="offset points",
#                 fontsize=8,  
#                 color="red"
#         )
        
#     # poipairs = connected_ltn_pairs | connected_other_pairs
#     poipairs = list(GT_abstract_nx.edges())
#     routenodepairs = [(u, v) for u, v in poipairs]

#     if debug:
#         print(f"Routing on network for investment level: {D} with routdenodepairs", routenodepairs)
    
#     GT_indices = set()
    
#     ## conditional routing 
#     # ltn --> ltn (all)
#     # ltn --> tess (all to one)
#     # tess --> tess (one to one)
#     # tess --> ltn (one to all)

#     GT_indices = set()
#     processed_pairs = set()

#     for u, v in routenodepairs:
#         poipair = (u, v)
#         if poipair in global_processed_pairs or tuple(reversed(poipair)) in global_processed_pairs:
#             continue
        
#         # Determine if nodes are neighbourhood or tessellation
#         is_u_neighbourhood = u in all_centroids['nearest_node'].values
#         is_v_neighbourhood = v in all_centroids['nearest_node'].values
        
#         if is_u_neighbourhood and is_v_neighbourhood:
#             # Both nodes are neighbourhoods
#             neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
#             neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
            
#             exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_a, 'osmid']
#             exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_b, 'osmid']
            
#             shortest_path_length, best_path = float('inf'), None
#             for ea in exit_points_a:
#                 for eb in exit_points_b:
#                     pair_id = tuple(sorted((ea, eb)))
#                     if pair_id in processed_pairs:
#                         continue
#                     processed_pairs.add(pair_id)
                    
#                     try:
#                         sp = nx.shortest_path(G_weighted, source=ea, target=eb, weight='length')
#                         sp_length = nx.shortest_path_length(G_weighted, source=ea, target=eb, weight='length')
#                         if sp_length < shortest_path_length:
#                             shortest_path_length, best_path = sp_length, sp
#                     except nx.NetworkXNoPath:
#                         continue
            
#             if best_path:
#                 GT_indices.update(best_path)

#         elif is_u_neighbourhood and not is_v_neighbourhood:
#             # Neighbourhood to Tessellation
#             neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == u, 'neighbourhood_id'].values[0]
#             exit_points_a = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']

#             shortest_path_length, best_path = float('inf'), None
#             for ea in exit_points_a:
#                 try:
#                     sp = nx.shortest_path(G_weighted, source=ea, target=v, weight='length')
#                     sp_length = nx.shortest_path_length(G_weighted, source=ea, target=v, weight='length')
#                     if sp_length < shortest_path_length:
#                         shortest_path_length, best_path = sp_length, sp
#                 except nx.NetworkXNoPath:
#                     continue
            
#             if best_path:
#                 GT_indices.update(best_path)

#         elif not is_u_neighbourhood and is_v_neighbourhood:
#             # Tessellation to Neighbourhood
#             neighbourhood_id = all_centroids.loc[all_centroids['nearest_node'] == v, 'neighbourhood_id'].values[0]
#             exit_points_b = exit_points.loc[exit_points['neighbourhood_id'] == neighbourhood_id, 'osmid']

#             shortest_path_length, best_path = float('inf'), None
#             for eb in exit_points_b:
#                 try:
#                     sp = nx.shortest_path(G_weighted, source=u, target=eb, weight='length')
#                     sp_length = nx.shortest_path_length(G_weighted, source=u, target=eb, weight='length')
#                     if sp_length < shortest_path_length:
#                         shortest_path_length, best_path = sp_length, sp
#                 except nx.NetworkXNoPath:
#                     continue
            
#             if best_path:
#                 GT_indices.update(best_path)

#         elif not is_u_neighbourhood and not is_v_neighbourhood:
#             # Tessellation to Tessellation
#             try:
#                 sp = nx.shortest_path(G_weighted, source=u, target=v, weight='length')
#                 GT_indices.update(sp)
#             except nx.NetworkXNoPath:
#                 continue

#         else:
#             print("This should never happen.")

#         global_processed_pairs.add(poipair) # track pairs that we've already routed between
    
#     cumulative_GT_indices.update(GT_indices) # add routes we've already calculated previously to current GT

#     # Generate subgraph for selected routes
#     GT = G_caralls[placeid].subgraph(cumulative_GT_indices)
#     #deweight_edges(GT, tag_lts)
#     # ensure weight attibute is stored
#     for u, v, data in GT.edges(data=True):
#         if 'length' in data:
#             data['weight'] = data['length']
            
#     GTs.append(GT)
#     GT_nodes, GT_edges = ox.graph_to_gdfs(GT)
#     GTs_gdf.append(GT_edges)


#     if debug:
#         GT_nodes, GT_edges = ox.graph_to_gdfs(GT)
#         GT_edges = GT_edges.to_crs(epsg=3857)
#         ax = GT_edges.plot()
#         ltn_gdf.plot(ax=ax, color='red', markersize=10)
#         tess_gdf.plot(ax=ax, color='green', markersize=5)
#         ax.set_title(f"Investment level: {D}, Number of edges: {len(GT.edges)}")


#     # write results
#     #results = {"placeid": placeid, "prune_measure": prune_measure, "poi_source": poi_source, "prune_quantiles": investment_levels, "GTs": GTs_igraph, "GT_abstracts": GT_abstracts_igraph}
#     #write_result(results, "pickle", placeid, poi_source, prune_measure, ".pickle", weighting=weighting)

        





In [ ]:
# iteration = 0

# ax = GTs_gdf[iteration].to_crs(epsg=3857).plot()
# GT_abstracts_gdf[iteration].to_crs(epsg=3857).plot(ax=ax, color='red')

# ltn_gdf.plot(ax=ax, color='red', markersize=10)
# tess_gdf.plot(ax=ax, color='green', markersize=5)

In [ ]:
# greedy_nx

# # Calculate edge betweenness centrality
# edge_betweenness = nx.edge_betweenness_centrality(greedy_nx, weight='sp_lts_distance')

# # Convert to GeoDataFrame
# edges = []
# for u, v, data in greedy_nx.edges(data=True):
#     edge_data = {
#         'start_osmid': u,
#         'end_osmid': v,
#         'geometry': data['geometry'],
#         'betweenness_centrality': edge_betweenness[(u, v)]
#     }
#     edges.append(edge_data)

# greedy_gdf_with_betweenness = gpd.GeoDataFrame(edges).set_crs(3857)

# # Calculate the rank of betweenness centrality
# greedy_gdf_with_betweenness['betweenness_rank'] = greedy_gdf_with_betweenness['betweenness_centrality'].rank(ascending=False).astype(int)

# greedy_gdf_with_betweenness

In [ ]:
# # Filter the top 20 rows based on betweenness_rank
# top_20_betweenness = greedy_gdf_with_betweenness.nsmallest(20, 'betweenness_rank')

# # Use .explore() to visualize the top 20 betweenness ranking
# top_20_betweenness.explore(cmap='viridis', column='betweenness_rank', legend=True)

In [ ]:
# greedy_gdf_with_betweenness.explore(cmap='viridis', column='betweenness_rank', legend=True)

In [ ]:
# iteration = 50
# m= GT_abstracts_gdf[iteration].to_crs(epsg=3857).explore(color='red')
# neighbourhoods['Newcastle Upon Tyne'].explore(m=m, color='red', markersize=10)

In [ ]:

# # Calculate total lengths for each graph in GTs
# total_lengths = []
# for G in GTs:
#     lengths = nx.get_edge_attributes(G, 'length')
#     total_length = sum(lengths.values())
#     total_lengths.append(total_length)

# # Plot the total lengths
# plt.figure(figsize=(10, 6))
# plt.plot(total_lengths, marker='o', linestyle='-')
# plt.xlabel('Graph Index')
# plt.ylabel('Total Length (meters)')
# plt.title('Total Length of Graphs in GTs')
# plt.grid(True)
# plt.show()

In [ ]:
# investment_levels

In [ ]:

# GT_nodes, GT_edges = ox.graph_to_gdfs(GT)
# GT_edges = GT_edges.to_crs(epsg=3857)
# ax = GT_edges.plot()

# GT_abs_nodes, GT_abs_edges = ox.graph_to_gdfs(GT_abs)
# GT_abs_edges = GT_abs_edges.to_crs(epsg=3857)
# GT_abs_edges.plot(ax=ax, color='red', linewidth=2)

# ltn_gdf.plot(ax=ax, color='red', markersize=10)
# tess_gdf.plot(ax=ax, color='green', markersize=5)
# ax.set_title(f"Investment level: {D}, Number of edges: {len(GT.edges)}")

In [ ]:
# gpkg_path = PATH["data"] + placeid + "/" + placeid + '_biketrack.gpkg'
# G_biketrack = ox_gpkg_to_graph(gpkg_path)
# G_biketrack.remove_nodes_from(list(nx.isolates(G_biketrack)))

In [ ]:
# import folium
# import geopandas as gpd

# # Assuming G_biketrack_edges and exit_points_3857 are already defined as GeoDataFrames

# # Ensure both GeoDataFrames are in the same CRS (e.g., EPSG:4326 for folium compatibility)
# if G_biketrack_edges.crs != 'EPSG:4326':
#     G_biketrack_edges = G_biketrack_edges.to_crs('EPSG:4326')
# if exit_points_3857.crs != 'EPSG:4326':
#     exit_points_3857 = exit_points_3857.to_crs('EPSG:4326')

# # Create a folium map centered on the first point of G_biketrack_edges
# map_center = [G_biketrack_edges.geometry.centroid.y.iloc[0], G_biketrack_edges.geometry.centroid.x.iloc[0]]
# m = folium.Map(location=map_center, zoom_start=14)

# # Add G_biketrack_edges to the map as a line layer
# for _, row in G_biketrack_edges.iterrows():
#     folium.PolyLine(
#         locations=[(point[1], point[0]) for point in row.geometry.coords],  # Swap lat/lon for folium
#         color='blue',
#         weight=2,
#         opacity=0.7
#     ).add_to(m)

# # Add exit_points_3857 to the map as a point layer
# for _, row in exit_points_3857.iterrows():
#     folium.CircleMarker(
#         location=(row.geometry.y, row.geometry.x),  # Swap lat/lon for folium
#         radius=5,
#         color='red',
#         fill=True,
#         fill_color='red',
#         fill_opacity=0.7
#     ).add_to(m)

# # Display the map
# m

In [ ]:
# ## investigate if exit points match with neighbourhoods?

# # Set up a larger figure
# fig, ax = plt.subplots(figsize=(20, 14))  # Adjust the width and height as needed


# # Add G_weighted edges
# G_weighted_nodes, G_weighted_edges = ox.graph_to_gdfs(G_weighted)
# G_weighted_edges = G_weighted_edges.to_crs(epsg=3857)  # Ensure CRS matches
# G_weighted_edges.plot(ax=ax, color='lightgrey', linewidth=0.5, alpha=0.8, zorder = 0)  # Light grey with thin linewidth

# # Add bike track edges
# G_biketrack_nodes, G_biketrack_edges = ox.graph_to_gdfs(G_biketrack)
# G_biketrack_edges = G_biketrack_edges.to_crs(epsg=3857)
# G_biketrack_edges.plot(ax=ax, color='turquoise', linewidth=0.5, alpha=0.8, zorder = 2)  # Light grey with thin linewidth


# # Plot the main graph and layers
# GT_nodes, GT_edges = ox.graph_to_gdfs(GT)
# GT_edges = GT_edges.to_crs(epsg=3857)
# GT_edges.plot(ax=ax, color='orange', zorder = 1)

# # plot exit points
# exit_points_3857 = exit_points.to_crs(epsg=3857)
# exit_points_3857.plot(ax=ax, color='red', markersize=3)




# # Remove x and y axis labels and ticks
# ax.axis('off')  # This removes the entire axis, including labels and ticks

# # Enhance plot aesthetics
# ax.set_title(f"Meters of investment: {D/10}")
# ax.legend(loc="upper left")

# # Show the plot
# plt.show()
